# FILE FOR SCRAPPING ARTISTS DATA INCLUDING NAME AGE, NATIONALITY, GENDER

**PROJET DALAS**

Hiba Akbi, Dwi Prima Putri

In [ ]:
!pip install beautifulsoup4

In [ ]:
import bs4
import lxml
import pandas as pd
import urllib
from bs4 import BeautifulSoup
from urllib import request

In [ ]:
import csv
def save_to_file(filename, header, data):
  with open(filename, mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(header)
      writer.writerows(data)


In [ ]:
#fucntion to read data from a csv file. return a list of lists.
def read_file(filename):
  with open(filename, "r", newline="", encoding="utf-8") as file:
      reader = csv.reader(file)
      header=next(reader)
      data = list(reader)  # Convert to list of rows
  return header, data

## **FIND UNIQUE ARTISTS AND THEIR METADATA**

I'm going to get all the unique artists from my song data. The purpose will be to get some insights on the artists (their name, nationality, age , gender). For this, I will be using Wikepedia.


First, lets get a unique table that has all our artists. We extract these from our songs table.

In [ ]:
header, artist_data= read_file("clean_single_songs_genius.csv")
artist_data_df= pd.DataFrame(artist_data, columns=header)

In [ ]:
artist_data_df

,Song ID,Title,Cleaned Title,Artist,Main Artist,Feat,Song Writers/Credits,Producer,Label,Album,Duration,Spotify Featured Artists,Spotify Found,Release Date,Genius Album,Genius Found
0,0,I Don't Want To Miss A Thing,I Don't Want To Miss A Thing,Aerosmith,Aerosmith,,D.E.Warren,M.Serletic,Columbia,I Don't Want To Miss A Thing EP,299.723,,1,1998-08-18,,1
1,1,The First Night,The First Night,Monica,Monica,,"J.Dupri Mauldin, T.Savage, M.McLeod, P.J.Sawyer",J.Dupri,Arista,The Boy Is Mine,235.573,,1,1998-07-07,,1
2,2,Crush,Crush,Jennifer Paige,Jennifer Paige,,"A.Goldmark, M.Mueller, B.Cosgrove, K.Clark","A.Goldmark, J.Bralower",edel/Hollywood,Jennifer Paige,200.293,,1,1998-06-16,,1
3,3,My Way,My Way,Usher,Usher,,"J.Dupri Mauldin, M.L.Seal, Jr., U.Raymond IV",J.Dupri,LaFace/Arista,My Way,214.853,,1,1997-09-16,,1
4,4,The Boy Is Mine,The Boy Is Mine,Brandy & Monica,Brandy,Monica,"R.Jerkins, B.R.Norwood, L.A.Daniels, F.Jerkins...","R.Jerkins, D.Austin, Brandy",Atlantic,the boy is mine (Remix),213.384,"Brandy,Monica",1,1998-05-19,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11446,135760,Passport Junkie,Passport Junkie,Rod Wave,Rod Wave,,"R.M.Green, T.Kozluklu, K.B.Vyacheslavovich, D....","T5, Exuising, Wonderyo",Alamo,Last Lap,147.749,,1,2024-09-14,,1
11447,135773,Talk Talk,Talk Talk,Charli xcx Featuring Troye Sivan,Charli xcx,Troye Sivan,"C.E.Aitchison, A.G.Cook, R.M.Birchard, T.S.Mel...","Hudson Mohawke, A.G.Cook",Atlantic,Brat and it’s completely different but also st...,173.076,Troye Sivan,1,2024-06-07,,1
11448,135774,Bad Dreams,Bad Dreams,Teddy Swims,Teddy Swims,,"J.C.Dimsdale, J.H.Ryan, J.C.Bunetta, M.D.Zara,...","J.H.Ryan, J.Bunetta, M.Zara",SWIMS Int./Warner,Bad Dreams,184.577,,1,2024-09-13,,1
11449,135794,I Never Lie,I Never Lie,Zach Top,Zach Top,,"Z.Top, C.Chamberlain, T.Nichols",C.Chamberlain,Leo33,Cold Beer & Country Music,224.906,,1,2024-04-05,,1


In [ ]:
artist_data_df =artist_data_df.drop_duplicates(subset=['Main Artist'], keep='first')
print("After deleting redundant artists, we have: ", artist_data_df.shape[0], " unique artists.")

artist_data_df= artist_data_df[['Main Artist','Spotify Found']]

artist_data_df

After deleting redundant artists, we have:  2303  unique artists.


,Main Artist,Spotify Found
0,Aerosmith,1
1,Monica,1
2,Jennifer Paige,1
3,Usher,1
4,Brandy,1
...,...,...
11432,Chase Matthew,1
11437,Tito Double P,1
11439,Addison Rae,1
11440,Surf Curse,1


We have 2303 unique artists. Let's try to get infos about them from wikepedia.

In [ ]:
pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=5cdddd2973157e5ccbcf61412656485619152bd7a3bf45b4d37e37bf9bfb30cf
  Stored in directory: /root/.cache/pip/wheels/0b/0f/39/e8214ec038ccd5aeb8c82b957289f2f3ab2251febeae5c2860
Successfully built wikipedia-api


## find artists info (name, wiki, borthday, nationality)

In [ ]:

# import wikipediaapi
# import requests

# def find_artists_data(artist_name):
#   page = wiki_wiki.page(artist_name)
#   print("try for",  artist_name)
#   if page.exists():
#       url= page.fullurl
#       print(url)
#       response = requests.get(url)
#       if response.status_code == 200:
#           soup = BeautifulSoup(response.text, "html.parser")
#           content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
#           # print(content)
#           if content==None: return 0, None, None, None, None #if the page was found but theires needs for clarification

#           #is it a group or a single artist
#           group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")
#           if group:
#             group= group.find_next_sibling()
#             group=group.find_all("li")

#             member=len(group)
#             name=None
#             birthday=None
#             nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
#             if nationality:
#               nationality= nationality.find_next_sibling()
#               nationality= nationality.text.split(",")[-1] if nationality else None

#           else: #not a group

#             member=1
#             #full name of the artist
#             name= content.find("div", class_="nickname")
#             name=name.text if name else None
#             #brithday of the artist
#             bday= content.find("span", class_="bday")
#             birthday=bday.text if bday else None
#             #birthplace of the artist
#             nationality= content.find("div", class_="birthplace")
#             nationality= nationality.text.split(",")[-1] if nationality else None
#       return 1, name, birthday, nationality, member

#   else: return 0, None, None, None, None



In [ ]:
import wikipediaapi
import requests

def find_artists_data(artist_name):
  page = wiki_wiki.page(artist_name)
  print("try for",  artist_name)
  if page.exists():
      url= page.fullurl
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return 0, None, None, None, None, None #if the page was found but theires needs for clarification

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")
          if group:
            group= group.find_next_sibling()
            group=group.find_all("li")

            member=len(group)
            name=None
            birthday=None
            nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
            if nationality:
              nationality= nationality.find_next_sibling()
              nationality= nationality.text.split(",")[-1] if nationality else None

          else: #not a group

            member=1
            #full name of the artist
            name= content.find("div", class_="nickname")
            name=name.text if name else None
            #brithday of the artist
            bday= content.find("span", class_="bday")
            birthday=bday.text if bday else None
            #birthplace of the artist
            nationality= content.find("div", class_="birthplace")
            nationality= nationality.text.split(",")[-1] if nationality else None
      return 1, name, birthday, nationality, member, url

  else: return 0, None, None, None, None, None



In [ ]:



i=0
found_wiki=[]
wiki_url=[]
birthdays=[]
nationalities=[]
names=[]
members=[]
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in artist_data_df.iterrows():
  i+=1
  artist_name=row["Main Artist"]
  print("*****************")
  print(i, artist_name)
  found, name, bday, nationality, member, url= find_artists_data(artist_name)
  #if the page wasnt found because on an mbiguity , try again with "singer"
  if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (singer)")
  if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (musician)")
  if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (rapper)")
  if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (group)")
  if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (band)")
  if found!=1: found=0 #convert it again to 0 to have entries in 0, 1 only
  found_wiki.append(found)
  names.append(name)
  birthdays.append(bday)
  nationalities.append(nationality)
  members.append(member)
  wiki_url.append(url)




Streaming output truncated to the last 5000 lines.
1253 Steel Magnolia
try for Steel Magnolia
https://en.wikipedia.org/wiki/Steel_Magnolia
*****************
1254 Sam Adams
try for Sam Adams
https://en.wikipedia.org/wiki/Samuel_Adams
try for Sam Adams (singer)
try for Sam Adams (musician)
https://en.wikipedia.org/wiki/Sammy_Adams
*****************
1255 Spose
try for Spose
https://en.wikipedia.org/wiki/Spose
*****************
1256 Jaron
try for Jaron
https://en.wikipedia.org/wiki/Jaron
try for Jaron (singer)
try for Jaron (musician)
try for Jaron (rapper)
try for Jaron (group)
try for Jaron (band)
*****************
1257 Diddy - Dirty Money
try for Diddy - Dirty Money
https://en.wikipedia.org/wiki/Diddy_%E2%80%93_Dirty_Money
*****************
1258 Travie McCoy
try for Travie McCoy
https://en.wikipedia.org/wiki/Travie_McCoy
*****************
1259 Mike Posner
try for Mike Posner
https://en.wikipedia.org/wiki/Mike_Posner
*****************
1260 Jerrod Niemann
try for Jerrod Niemann
https://en

In [ ]:
print(len(found_wiki))
print(len(wiki_url))
print(len(names))
print(len(nationalities))
print(len(birthdays))
print(len(members))



2303
2303
2303
2303
2303
2303


In [ ]:
artist_data_df["Full Name"]= names
artist_data_df["Country"]= nationalities
artist_data_df["Birth Date"]= birthdays
artist_data_df["Group Members"]= members
artist_data_df["Found Wiki"]=found_wiki
artist_data_df["Wiki URL"]= wiki_url
artist_data_df

,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,Aerosmith,1,None,U.S.,None,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,Jennifer Paige,1,None,None,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...
11432,Chase Matthew,1,None,None,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
11437,Tito Double P,1,None,None,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
11440,Surf Curse,1,None,U.S.,None,0.0,1,https://en.wikipedia.org/wiki/Surf_Curse


Let's look into our data, see if there's any patterns in the rows where wikipedia couldnt find the page.

In [ ]:
df= artist_data_df[artist_data_df["Found Wiki"]==0]
len(df)


183

In [ ]:
artist_data_df.to_csv('artist_data.csv', index=True)

In [ ]:
import pandas as pd

artist_data_df = pd.read_csv("artist_data (1).csv")
artist_data_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
2301,11440,Surf Curse,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/Surf_Curse


In [ ]:
df= artist_data_df[artist_data_df["Found Wiki"]==0]
len(df)

183

On a quand meme 183 artiste sur 2303 manquants. Pas super.

We notice that we got some extra keywords we can try to try and get smthng from wikepeida. For instance, (singer-songwriter), (group).
We also notice that while the capital letters often help in distinguishing between artists, in this case, we ight want to try a lowercase.

In [ ]:
artist_name="H.A (Hiba Akbi)"
if "(" in artist_name and ")" in artist_name:
    print(artist_name)
    s= artist_name.find("(")
    e= artist_name.find(")")
    artist_name= artist_name[s+1:e]
    print(artist_name)

H.A (Hiba Akbi)
Hiba Akbi


In [ ]:
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in artist_data_df.iterrows():
  i+=1
  if row["Found Wiki"]==0: #retry for
    print(i)
    artist_name=row["Main Artist"].title()
    if "(" in artist_name and ")" in artist_name: #get the full name of the artist which could be between arenthesis
      s= artist_name.find("(")
      e= artist_name.find(")")
      artist_name= artist_name[s+1:e]
    print("*****************")
    print(i, artist_name)
    found, name, bday, nationality, member, url= find_artists_data(artist_name)
    #if the page wasnt found because on an mbiguity , try again with "singer"
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (singer)")
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (musician)")
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (singer-songwriter)")
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (rapper)")
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (group)")
    if found!=1: found, name, bday, nationality, member, url= find_artists_data(artist_name+" (band)")


    if found==1: #if found, now update
      artist_data_df.at[idx, "Found Wiki"] = found
      artist_data_df.at[idx, "Full Name"] = name
      artist_data_df.at[idx, "Birth Date"] = bday
      artist_data_df.at[idx, "Country"] = nationality
      artist_data_df.at[idx, "Group Members"] = member
      artist_data_df.at[idx, "Wiki URL"] = url




9
*****************
9 Next
try for Next
https://en.wikipedia.org/wiki/Next
try for Next (singer)
try for Next (musician)
try for Next (singer-songwriter)
try for Next (rapper)
try for Next (group)
https://en.wikipedia.org/wiki/Next
try for Next (band)
https://en.wikipedia.org/wiki/Next
23
*****************
23 Nicole
try for Nicole
https://en.wikipedia.org/wiki/Nicole
try for Nicole (singer)
https://en.wikipedia.org/wiki/Nicole
try for Nicole (musician)
try for Nicole (singer-songwriter)
try for Nicole (rapper)
try for Nicole (group)
try for Nicole (band)
25
*****************
25 Voices Of Theory
try for Voices Of Theory
try for Voices Of Theory (singer)
try for Voices Of Theory (musician)
try for Voices Of Theory (singer-songwriter)
try for Voices Of Theory (rapper)
try for Voices Of Theory (group)
try for Voices Of Theory (band)
46
*****************
46 Jagged Edge
try for Jagged Edge
https://en.wikipedia.org/wiki/Jagged_Edge
try for Jagged Edge (singer)
try for Jagged Edge (musician)
t

In [ ]:
df= artist_data_df[artist_data_df["Found Wiki"]==0]
print("Number of artists not found on wikepdia: ", len(df))


Number of artists not found on wikepdia:  157


In [ ]:
artist_data_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
2301,11440,Surf Curse,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/Surf_Curse


In [ ]:
artist_data_df.to_csv('artist_data_fixed.csv', index=False)

In [ ]:
import pandas as pd
old_df = pd.read_csv("artist_data_fixed.csv")
old_df #2303x9

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
2301,11440,Surf Curse,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/Surf_Curse


We do however notice somethign a bit odd in our data. Looking at "Group Members", we see that some have a '0' as their value. This is a flag that soemthign went wrong when scrapping data for a group.


In [ ]:
df= old_df[(old_df["Found Wiki"]==1)&(old_df["Group Members"]==0)]
print("Number of members found with 0 members: ", len(df))

Number of members found with 0 members:  114


We have 114 group members=0, let's fix that.

In [ ]:
#fucntion thta fixes the stuff for the group member =0
def find_artists_group(artist_name):
  page = wiki_wiki.page(artist_name)
  print("try for",  artist_name)
  if page.exists():
      found=0
      url= page.fullurl
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return 1, None #if the page was found but theires needs for clarification
          found=1

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")

          if group:
            group= group.find_next_sibling("td", {"class":"infobox-data"})
            if group:
              group=group.find_all("a")
              member=len(group)
              print(idx,member)
              return 1, member
          return 1, None
      return 0, None
  return 0,None

In [ ]:
import wikipediaapi
import requests
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in old_df.iterrows():
  i+=1
  if row["Found Wiki"]==1 and row["Group Members"]==0: #if group members =0
    artist_name=row["Main Artist"]
    if "(" in artist_name and ")" in artist_name: #get the full name of the artist which could be between arenthesis
      s= artist_name.find("(")
      e= artist_name.find(")")
      artist_name= artist_name[s+1:e]
    print("*****************")
    print(i, artist_name)
    found, member= find_artists_group(artist_name)
    #if the page wasnt found because on an mbiguity , try again with "singer"
    if found!=1:found, member= find_artists_group(artist_name+" (singer)")
    if found!=1:found, member= find_artists_group(artist_name+" (musician)")
    if found!=1:found, member= find_artists_group(artist_name+" (singer-songwriter)")
    if found!=1:found, member= find_artists_group(artist_name+" (rapper)")
    if found!=1:found, member= find_artists_group(artist_name+" (group)")
    if found!=1:found, member= find_artists_group(artist_name+" (band)")


    assert(found==1) #normalement on les retrouve toutes vu qu'on travaille sur des wiki_found=1
    if(member==0): print("Still an Issue of member=0 with:", row)
    elif found==1 and member!=0: old_df.loc[idx, "Group Members"] = member



*****************
44 Marcy Playground
try for Marcy Playground
https://en.wikipedia.org/wiki/Marcy_Playground
43 3
*****************
53 69 Boyz
try for 69 Boyz
https://en.wikipedia.org/wiki/69_Boyz
52 0
Still an Issue of member=0 with: Unnamed: 0                                          55
Main Artist                                    69 Boyz
Spotify Found                                        1
Full Name                                          NaN
Country                                           U.S.
Birth Date                                         NaN
Group Members                                      0.0
Found Wiki                                           1
Wiki URL         https://en.wikipedia.org/wiki/69_Boyz
Name: 52, dtype: object
*****************
72 Imajin
try for Imajin
https://en.wikipedia.org/wiki/Imajin
71 2
*****************
80 Myron
try for Myron
https://en.wikipedia.org/wiki/Myron
*****************
102 Nastyboy Klick
try for Nastyboy Klick
https://en.wikipedia.or

Let's see if this fixes anything

In [ ]:
df= old_df[(old_df["Found Wiki"]==1)&(old_df["Group Members"]==0)]
print("Number of members found with 0 members: ", len(df))

Number of members found with 0 members:  51


In [ ]:
df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
52,55,69 Boyz,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/69_Boyz
101,106,Nastyboy Klick,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/NB_Ridaz
138,151,Reel Tight,1,NaN,Tennessee,NaN,0.0,1,https://en.wikipedia.org/wiki/Reel_Tight
224,275,Sawyer Brown,1,NaN,United States,NaN,0.0,1,https://en.wikipedia.org/wiki/Sawyer_Brown
257,342,Sporty Thievz,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/Sporty_Thievz
268,369,Blessid Union Of Souls,1,NaN,United States,NaN,0.0,1,https://en.wikipedia.org/wiki/Blessid_Union_of...
274,385,The Beatnuts,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/The_Beatnuts
279,394,Tanto Metro,1,NaN,Jamaica,NaN,0.0,1,https://en.wikipedia.org/wiki/Tanto_Metro_and_...
386,663,Nine Days,1,NaN,U.S.,NaN,0.0,1,https://en.wikipedia.org/wiki/Nine_Days
400,706,Ruff Endz,1,NaN,United States,NaN,0.0,1,https://en.wikipedia.org/wiki/Ruff_Endz


On est passe de 114 artist a 50 avec ce probleme. On explore un peu plus pour voit si c'est posssible de gerer ca. On voit justement que c'est un probleme avce les tags, parfois eu lieu de < a> on a < br>. donc on gere ca.

In [ ]:
#fucntion thta fixes the stuff for the group member =0
def find_artists_group(artist_name):
  page = wiki_wiki.page(artist_name)
  print("try for",  artist_name)
  if page.exists():
      found=0
      url= page.fullurl
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return 1, None #if the page was found but theires needs for clarification
          found=1

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")

          if group:
            group= group.find_next_sibling("td", {"class":"infobox-data"})
            if group:
              group=group.find_all("br")
              member=len(group)+1
              print(idx,member)
              return found, member
          return found, None
      return 0, None
  return 0,None

In [ ]:
import wikipediaapi
import requests
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in old_df.iterrows():
  i+=1
  if row["Found Wiki"]==1 and row["Group Members"]==0: #if group members =0
    artist_name=row["Main Artist"]
    if "(" in artist_name and ")" in artist_name: #get the full name of the artist which could be between arenthesis
      s= artist_name.find("(")
      e= artist_name.find(")")
      artist_name= artist_name[s+1:e]
    print("*****************")
    print(i, artist_name)
    found, member= find_artists_group(artist_name)
    #if the page wasnt found because on an mbiguity , try again with "singer"
    if found!=1:found, member= find_artists_group(artist_name+" (singer)")
    if found!=1:found, member= find_artists_group(artist_name+" (musician)")
    if found!=1:found, member= find_artists_group(artist_name+" (singer-songwriter)")
    if found!=1:found, member= find_artists_group(artist_name+" (rapper)")
    if found!=1:found, member= find_artists_group(artist_name+" (group)")
    if found!=1:found, member= find_artists_group(artist_name+" (band)")


    assert(found==1) #normalement on les retrouve toutes vu qu'on travaille sur des wiki_found=1
    if(member==0): print("Issue of member=0 with:", row)
    elif found==1 and member!=0: old_df.loc[idx, "Group Members"] = member



*****************
53 69 Boyz
try for 69 Boyz
https://en.wikipedia.org/wiki/69_Boyz
52 2
*****************
102 Nastyboy Klick
try for Nastyboy Klick
https://en.wikipedia.org/wiki/NB_Ridaz
101 1
*****************
139 Reel Tight
try for Reel Tight
https://en.wikipedia.org/wiki/Reel_Tight
138 4
*****************
225 Sawyer Brown
try for Sawyer Brown
https://en.wikipedia.org/wiki/Sawyer_Brown
224 4
*****************
258 Sporty Thievz
try for Sporty Thievz
https://en.wikipedia.org/wiki/Sporty_Thievz
257 2
*****************
269 Blessid Union Of Souls
try for Blessid Union Of Souls
https://en.wikipedia.org/wiki/Blessid_Union_of_Souls
268 5
*****************
275 The Beatnuts
try for The Beatnuts
https://en.wikipedia.org/wiki/The_Beatnuts
274 2
*****************
280 Tanto Metro
try for Tanto Metro
https://en.wikipedia.org/wiki/Tanto_Metro_and_Devonte
279 2
*****************
387 Nine Days
try for Nine Days
https://en.wikipedia.org/wiki/Nine_Days
386 5
*****************
401 Ruff Endz
try for Ruff 

In [ ]:
df= old_df[(old_df["Found Wiki"]==1)&(old_df["Group Members"]==0)]
print("Number of members found with 0 members: ", len(df))

Number of members found with 0 members:  0


Bah super, on a regle le probleme.

## GET URL

getting the wikepedia url of the artists because it was dumb of me to not collect it as data earlier and i'll need it later so we run this again.

In [ ]:
print(len(urls))

2303


In [ ]:
old_df["Wiki URL"]=urls
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
2301,11440,Surf Curse,1,NaN,U.S.,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse


In [ ]:
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer)
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician)
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood
...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew
2299,11437,Tito Double P,1,NaN,NaN,1981-06-17,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae
2301,11440,Surf Curse,1,NaN,U.S.,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse


In [ ]:
print(10*"*")
print(len(old_df[old_df["Found Wiki"]==0]))
print(len(old_df[old_df["Wiki URL"].isna()]))
#shoudl be equal

**********
183
183


In [ ]:
old_df.to_csv('artist_ya_plus_de_pb.csv', index=False)

## ETHINICITY AND GENDER
Now one really annoying thing is that wikepedia doesn't explicitly state the gender and ethnicity of artists. So we'll have to go about it in a way that is a bit creative. At the end of the wikepedia page, we have something like "subcategories" that an artist page can belong to.

For exmaple, for "Lady Gaga", we find subcategories like "LGBTQ Singers", "American Singers with Italian Descent", "21st Century American Female Singers".

So we'll go get our infos from there.

We identify the following:
*   21st-century American/Canadian.... women/male singers

For ethnicity, it's more complicated because all americans be tagged under "european descent" of some sort. When, they are in fact just white (we don't realy care about if Billie Eilish has irish descent from her great-grandfather, but we do care if Kanye West is of African-american descent)

So let's first handle the gender.


### **Search for gender**

In [ ]:
#return flag found and the gender. if flag=0, then gender = None
#returns flag found gender

def find_gender(artists_url):
  response = requests.get(artists_url)
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, "html.parser")
      tags= soup.find("div", id="mw-normal-catlinks")
      if tags==None: return 1, None #found the artist but didnt find the gender container
      tags=tags.find_all("li")
      for li in tags:
        if ("women" in li.get_text().lower()): return 1, "F"
        elif("male" in li.get_text().lower()): return 1, "M"
      return 1, None
  return 0, None


In [ ]:
genders=[]
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in old_df.iterrows():
  i+=1
  print(i)
  gender=None
  if row["Found Wiki"]==1 and row["Wiki URL"] and row["Group Members"]==1 : #of course, we only look for 1 singer not for groups
    found, gender= find_gender(row["Wiki URL"])
    if gender==None: print("Issue with dteermining gender for\n", row)
  genders.append(gender)

Streaming output truncated to the last 5000 lines.
Wiki URL         https://en.wikipedia.org/wiki/R.E.M.
Name: 131, dtype: object
133
134
135
Issue with dteermining gender for
 Unnamed: 0                                          146
Main Artist                             Mo Thugs Family
Spotify Found                                         1
Full Name                                           NaN
Country                                             NaN
Birth Date                                          NaN
Group Members                                       1.0
Found Wiki                                            1
Wiki URL         https://en.wikipedia.org/wiki/Mo_Thugs
Name: 134, dtype: object
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
Issue with dteermining gender for
 Unnamed: 0                                             174
Main Artist                                         Brooks
Spotify Found                                            

In [ ]:
len(genders)

2303

In [ ]:
old_df["Gender"]=genders
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,None
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F
...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F
2301,11440,Surf Curse,1,NaN,U.S.,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,None


In [ ]:
print("Number of artists with missing gender: ",len(old_df[old_df["Gender"].isna()]))

print("Out of those we have wiki urls for:",len(old_df[(old_df["Gender"].isna()) &(old_df["Found Wiki"]==1)]) )

Number of artists with missing gender:  870
Out of those we have wiki urls for: 713


Ok, so we have 713 artists we couldn't extract the genders for, by looking at the examples, some of those where groups (they had memebrs==1 because the members wheren't availabel on wikiepdia). We will decide to not proceed further as the most famous artist (likely grammy winners and nominnees) would surely be listed in wikepedia's subcategories. So we will ditch the rest.


### **Search for ethnicity**
Now we will look at ethnicity, this is likely to be harder to determine.
We will go with the followign categories:



*   Black/ African- American
*   Hispanic
*   Asian




In [ ]:
import pandas as pd

In [ ]:
old_df.to_csv('artist_before_country.csv', index=False)

In [ ]:
import pandas as pd

old_df = pd.read_csv("artist_before_country.csv")
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F
...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F
2301,11440,Surf Curse,1,NaN,U.S.,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN


##  **Some cleaning**
We ahve some cleaning to do, when it comes to countries/nationalities specifically.

Let's fix that

In [ ]:
old_df['Country'].unique()

array([' U.S.', nan, ' Canada', ' England', ' Sweden', ' US', ' Germany',
       ' Switzerland', ' Haiti', ' Puerto Rico', 'Germany[1][2]',
       ' Tennessee', ' Australia', ' U.S.[1]', ' Ireland',
       ' United States', ' U.S.[2]', ' Jamaica', ' Netherlands',
       ' Australia[1]', ' Nebraska', 'South Korea', 'Jamaica', ' Spain',
       ' Norway', ' Italy', 'Puerto Rico', ' USA', ' California', ' NY',
       ' New Zealand', ' California[1]', ' South Korea', ' Nigeria',
       ' Bolivia', ' Brazil', 'Belgium', ' U.S.[1][2]', ' Colombia',
       ' Mexico', ' England[1]', ' South Africa', 'France', ' U.S',
       ' Massachusetts U.S.', ' Georgia', 'Netherlands', ' Scotland',
       'Dominican Republic', ' U.S.[a]', ' Belgium', ' Barbados',
       ' Greece', ' New York[1]', ' U.K.', '  US', ' Soviet Union',
       ' Lebanon', ' Iceland', ' France', ' Finland', ' India',
       ' Ontario Canada', ' Missouri', ' UK', ' West Germany',
       ' Trinidad and Tobago', 'Australia', ' Romania

We see that we have lots of unclena data. We have U.S, Unites States, U.S. , USA , Massachusetts US...etc . Also we have England, United Kingdom, UK, ....etc

In [ ]:
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
0,0,Aerosmith,1,NaN,U.S.,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,None
1,1,Monica,1,Monica Denise Arnold,U.S.,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F
3,3,Usher,1,Usher Raymond IV[1][2],U.S.,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M
4,4,Brandy,1,Brandy Rayana Norwood,U.S.,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F
...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M
2299,11437,Tito Double P,1,NaN,NaN,1981-06-17,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M
2300,11439,Addison Rae,1,Addison Rae Easterling,U.S.,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F
2301,11440,Surf Curse,1,NaN,U.S.,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,None


In [ ]:
us_vars=["U.S.", "USA","U.S","US", "Missouri","Massachusetts U.S.","Georgia", "Wyoming","Nebraska","Pennsylvania","California","Tennessee","NY","New York"]
uk_vars=["England","U.K.","UK"]


In [ ]:
#truncate

old_df["Country"] = old_df["Country"].str.replace(r"\[.*?\]", "", regex=True) #enelever les []
old_df["Country"] = old_df["Country"].apply(lambda x: x.strip() if isinstance(x, str) else x)

old_df["Country"]= old_df["Country"].apply(lambda x: "United States" if isinstance(x, str) and x in us_vars else x)
old_df["Country"]= old_df["Country"].apply(lambda x: "United Kingdom" if isinstance(x, str) and x in uk_vars else x)
old_df['Country'].unique()

# mask_us=
# old_df["Country"] = old_df["Country"].apply(lambda x: x.strip())
# old_df['Country'].unique()
# mask=(clean_single_songs_df["Spotify Found"]=='0') & (clean_single_songs_df["Main Artist"].str.contains(r'\$', na=False))
# clean_single_songs_df.loc[mask, "Main Artist"] = clean_single_songs_df.loc[mask, "Main Artist"].replace(r'\$', 's', regex=True)

array(['United States', nan, 'Canada', 'United Kingdom', 'Sweden',
       'Germany', 'Switzerland', 'Haiti', 'Puerto Rico', 'Australia',
       'Ireland', 'Jamaica', 'Netherlands', 'South Korea', 'Spain',
       'Norway', 'Italy', 'New Zealand', 'Nigeria', 'Bolivia', 'Brazil',
       'Belgium', 'Colombia', 'Mexico', 'South Africa', 'France',
       'Scotland', 'Dominican Republic', 'Barbados', 'Greece',
       'Soviet Union', 'Lebanon', 'Iceland', 'Finland', 'India',
       'Ontario Canada', 'West Germany', 'Trinidad and Tobago', 'Romania',
       'SFR Yugoslavia(now Kosovo)', 'Isle of Man', 'Vietnam', 'Denmark',
       'Japan', 'Cuba', 'China', 'Philippines', 'Thailand', 'Argentina',
       'Nigeria.'], dtype=object)

In [ ]:
old_df

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
0,0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,None
1,1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F
3,3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M
4,4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F
...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M
2299,11437,Tito Double P,1,NaN,NaN,1981-06-17,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M
2300,11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F
2301,11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,None


In [ ]:
old_df.rename(columns={"Unnamed: 0": "ArtistID"}, inplace=True)

In [ ]:
old_df.to_csv('artist_data_fixed.csv', index=False)

## Clean this mess and explore what there is and what there isnt

Next run just  do:

In [ ]:
import pandas as pd

artists_df = pd.read_csv("artist_data_fixed.csv")


In [ ]:
artists_df #2303 x 10

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
0,0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN
1,1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F
3,3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M
4,4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F
...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M
2300,11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F
2301,11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN


In [ ]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ArtistID       2303 non-null   int64  
 1   Main Artist    2303 non-null   object 
 2   Spotify Found  2303 non-null   int64  
 3   Full Name      492 non-null    object 
 4   Country        989 non-null    object 
 5   Birth Date     1512 non-null   object 
 6   Group Members  2128 non-null   float64
 7   Found Wiki     2303 non-null   int64  
 8   Wiki URL       2146 non-null   object 
 9   Gender         1433 non-null   object 
dtypes: float64(1), int64(3), object(6)
memory usage: 180.1+ KB


In [ ]:
artists_df.apply(pd.Series.unique)

,0
ArtistID,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
Main Artist,"[Aerosmith, Monica, Jennifer Paige, Usher, Bra..."
Spotify Found,"[1, 0]"
Full Name,"[nan, Monica Denise Arnold, Usher Raymond IV[1..."
Country,"[United States, nan, Canada, United Kingdom, S..."
Birth Date,"[nan, 1980-10-24, 1973-09-03, 1978-10-14, 1979..."
Group Members,"[5.0, 1.0, nan, 2.0, 3.0, 4.0, 6.0, 11.0, 7.0,..."
Found Wiki,"[1, 0]"
Wiki URL,"[https://en.wikipedia.org/wiki/Aerosmith, http..."
Gender,"[nan, F, M]"


In [ ]:
artists_df["Gender"].unique()
artists_df["Gender"].value_counts(dropna=False)

,count
Gender,
M,973
NaN,870
F,460


In [ ]:
artists_df["Found Wiki"].unique()
artists_df["Found Wiki"].value_counts(dropna=False)

,count
Found Wiki,
1,2146
0,157


In [ ]:
artists_df["Group Members"].unique()
artists_df["Group Members"].value_counts(dropna=False)

,count
Group Members,
1.0,1705
NaN,175
4.0,118
2.0,105
3.0,88
5.0,78
6.0,17
7.0,6
9.0,5


In [ ]:
print("Number of artists with missing gender: ",len(artists_df[artists_df["Gender"].isna()]))
print("Number of artists with missing gender because thye aint on wikipedia: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==0) ]))
print("Number of artists with missing gender because thye a group: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"]>1) ]))
print("Number of artists with missing gender because we couldnt find it: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"]==1) ]))
print("Number of artists with missing gender because thye a group and we couldnt retrive infos?: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"].isna()) ]))

Number of artists with missing gender:  870
Number of artists with missing gender because thye aint on wikipedia:  157
Number of artists with missing gender because thye a group:  423
Number of artists with missing gender because we couldnt find it:  272
Number of artists with missing gender because thye a group and we couldnt retrive infos?:  18


In [ ]:
artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"].isna()) ]

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender
79,83,Myron,1,NaN,Switzerland,NaN,NaN,1,https://en.wikipedia.org/wiki/Myron_(duo),NaN
170,193,Everything,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Everything_(band),NaN
182,209,Cake,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Cake_(band),NaN
183,210,Fastball,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Fastball_(band),NaN
223,274,Orgy,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Orgy_(band),NaN
228,281,Silk,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Silk_(group),NaN
330,507,M2M,1,NaN,Norway,NaN,NaN,1,https://en.wikipedia.org/wiki/M2M_(band),NaN
340,534,Guy,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Guy_(band),NaN
422,758,Zombie Nation,1,NaN,Germany,NaN,NaN,1,https://en.wikipedia.org/wiki/Zombie_Nation_(m...,NaN
426,777,Changing Faces,1,NaN,United States,NaN,NaN,1,https://en.wikipedia.org/wiki/Changing_Faces_(...,NaN


In [ ]:
artists_df["Country"].unique()
artists_df["Country"].value_counts(dropna=False)

,count
Country,
NaN,1314
United States,712
United Kingdom,83
Canada,32
South Korea,19
Australia,17
Sweden,11
Mexico,11
Puerto Rico,10


In [ ]:
artists_df.loc[artists_df["Country"] == "West Germany", "Country"] = "Germany"
artists_df.loc[artists_df["Country"] == "Soviet Union", "Country"] = "Russia"
artists_df.loc[artists_df["Country"] == "Nigeria.", "Country"] = "Nigeria"
artists_df.loc[artists_df["Country"] == "Ontario Canada", "Country"] = "Canada"
artists_df["Country"].value_counts(dropna=False)

,count
Country,
NaN,1314
United States,712
United Kingdom,83
Canada,33
South Korea,19
Australia,17
Sweden,11
Mexico,11
Puerto Rico,10


### clean artist names in new column

In [ ]:
import re
import unicodedata
import string

def clean_parenthesis(title):
    parenthesis_pattern = r"\(.*?\)"
    title = re.sub(parenthesis_pattern, "", title)  # Replace all occurrences of parentheses
    return title.strip()

def clean_brackets(title):
    parenthesis_pattern = r"\[.*?\]"
    title = re.sub(parenthesis_pattern, "", title)  # Replace all occurrences of brackets
    return title.strip()

def clean_artist_name(alltxts):
  alltxts= unicodedata.normalize('NFD', alltxts).encode('ascii', 'ignore').decode('utf-8') #delet accents
  alltxts=re.sub("\$","s",alltxts) #fix $ issue
  if ("(") in alltxts: alltxts=clean_parenthesis(alltxts)
  if ("[") in alltxts: alltxts=clean_brackets(alltxts)
  alltxts = re.sub(r'[^a-zA-ZÀ-ÿ0-9\s]', '', alltxts) #delete special symbols but not accents
  punc = string.punctuation #delete punctuation
  punc += '\n\r\t'
  alltxts=alltxts.translate(str.maketrans(punc, ' ' * len(punc)))
  alltxts=re.sub(' +', ' ', alltxts)
  alltxts=alltxts.strip()
  return alltxts.title()

In [ ]:
artists_df["Cleaned Main Artist"] = artists_df["Main Artist"].astype(str).apply(lambda x: clean_artist_name(x))
artists_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M,Chase Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M,Tito Double P
2300,11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F,Addison Rae
2301,11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN,Surf Curse


In [ ]:
artists_df.to_csv('artist_data_fixed_final.csv', index=False)

In [ ]:
#delete duplicated artists based on the url, from 2303 to 2256 artists

artists_df_2 = artists_df[artists_df['Wiki URL'].isna()| ~artists_df.duplicated(subset='Wiki URL')]
artists_df_2

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M,Chase Matthew
2299,11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M,Tito Double P
2300,11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F,Addison Rae
2301,11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN,Surf Curse


In [ ]:
dropped_rows = artists_df[~(artists_df['Wiki URL'].isna() | ~artists_df.duplicated(subset='Wiki URL'))]
dropped_rows

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
124,135,JAY-Z,1,Shawn Corey Carter,United States,1969-12-04,1.0,1,https://en.wikipedia.org/wiki/Jay-Z,M,Jayz
281,401,LFO,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/LFO_(American_band),NaN,Lfo
368,620,NAS,1,Nasir bin Olu Dara Jones,United States,1973-09-14,1.0,1,https://en.wikipedia.org/wiki/Nas,M,Nas
380,653,matchbox twenty,1,NaN,United States,NaN,4.0,1,https://en.wikipedia.org/wiki/Matchbox_Twenty,NaN,Matchbox Twenty
467,897,Musiq Soulchild,1,NaN,NaN,1977-09-16,1.0,1,https://en.wikipedia.org/wiki/Musiq_Soulchild,M,Musiq Soulchild
517,1026,P. Diddy,1,Sean John Combs,United States,1969-11-04,1.0,1,https://en.wikipedia.org/wiki/Sean_Combs,M,P Diddy
556,1159,NB Ridaz,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/NB_Ridaz,NaN,Nb Ridaz
591,1264,N.O.R.E.,1,NaN,NaN,1977-09-06,1.0,1,https://en.wikipedia.org/wiki/N.O.R.E.,M,Nore
606,1298,Lil Wayne,1,Dwayne Michael Carter Jr.,United States,1982-09-27,1.0,1,https://en.wikipedia.org/wiki/Lil_Wayne,M,Lil Wayne
692,1609,Bow Wow,1,NaN,NaN,1987-03-09,1.0,1,https://en.wikipedia.org/wiki/Bow_Wow_(rapper),M,Bow Wow


ok donc la on a tt clean au niveau des artistes, on doit juste apsser uen deucieme sur les doublons avant d e creer la liste finale des artistes.


In [ ]:
artists_df_2 = artists_df_2.drop([1307, 1351, 1410])

In [ ]:
artists_df_2["ArtistID"] = artists_df_2["ArtistID"].astype(str).apply(lambda x: "A" + x)

In [ ]:
artists_df_2.to_csv('artist_data_final.csv', index=False)

## To load next time

In [ ]:
import pandas as pd

artists_df = pd.read_csv("artist_data_final.csv")

In [ ]:
artists_df #2253x11

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2248,A11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M,Chase Matthew
2249,A11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M,Tito Double P
2250,A11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F,Addison Rae
2251,A11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN,Surf Curse


well we do have a lot of missing data for our countries, not evry nice.

In [ ]:
print(len(artists_df[artists_df['Country'].isna()]))

1295


In [ ]:
import wikipediaapi
import requests


def find_artists_country(url):
  if url:
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return None

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")
          if group: return None
          else: #not a group

            #birthplace of the artist
            nationality= content.find("div", class_="birthplace")
            if nationality:
              nationality= nationality.text.split(",")[-1]
            else:
              nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
              if nationality:
                nationality= nationality.find_next_sibling()
                nationality= nationality.text.split(",")[-1] if nationality else None
              else: nationality=None

            if nationality==None:
              born= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Born")
              if born:
                nationality= born.find_next_sibling() #next element thtc otnains the indo
                if nationality : nationality = nationality.find_all(recursive=False) #all children
                if len(nationality)==0: return None
                if nationality : nationality= nationality[-1] if nationality else None #last child
                print(nationality)
                if nationality: nationality = nationality.next_sibling.strip().split(",")[-1] if nationality.next_sibling else nationality.text.strip().split(",")[-1]
              else: nationality=None
      if nationality!= None: return nationality

  return None



In [ ]:
filtered_df = artists_df[~artists_df['Wiki URL'].isna() & artists_df['Country'].isna()]
print(len(filtered_df))
filtered_df

1141


,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
2,A2,Jennifer Paige,1,NaN,NaN,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
5,A5,Inoj,1,NaN,NaN,1976-11-27,1.0,1,https://en.wikipedia.org/wiki/INOJ,F,Inoj
6,A6,Edwin McCain,1,NaN,NaN,1970-01-20,1.0,1,https://en.wikipedia.org/wiki/Edwin_McCain,M,Edwin Mccain
11,A11,Mase,1,NaN,NaN,1975-08-27,1.0,1,https://en.wikipedia.org/wiki/Mase,M,Mase
12,A12,Faith Hill,1,NaN,NaN,1967-09-21,1.0,1,https://en.wikipedia.org/wiki/Faith_Hill,F,Faith Hill
...,...,...,...,...,...,...,...,...,...,...,...
2246,A11396,Drew Baldridge,1,NaN,NaN,1991-07-29,1.0,1,https://en.wikipedia.org/wiki/Drew_Baldridge,M,Drew Baldridge
2247,A11416,Gigi Perez,1,NaN,NaN,2000-02-04,1.0,1,https://en.wikipedia.org/wiki/Gigi_Perez,NaN,Gigi Perez
2248,A11432,Chase Matthew,1,NaN,NaN,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M,Chase Matthew
2249,A11437,Tito Double P,1,NaN,NaN,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M,Tito Double P


In [ ]:
i=0
for idx, row in filtered_df.iterrows():
  i+=1
  print(i)
  nationality= find_artists_country(row['Wiki URL'])
  if nationality!=None:
    artists_df.loc[idx, 'Country']= nationality
    print(nationality, "\n", row)


Streaming output truncated to the last 5000 lines.
Birth Date                                           1992-05-19
Group Members                                               1.0
Found Wiki                                                    1
Wiki URL               https://en.wikipedia.org/wiki/Marshmello
Gender                                                      NaN
Cleaned Main Artist                                  Marshmello
Name: 1713, dtype: object
821
https://en.wikipedia.org/wiki/A_Boogie_wit_da_Hoodie
<a href="/wiki/The_Bronx" title="The Bronx">The Bronx</a>
 U.S. 
 ArtistID                                                           A6762
Main Artist                                       A Boogie Wit da Hoodie
Spotify Found                                                          1
Full Name                                                            NaN
Country                                                              NaN
Birth Date                                         

In [ ]:
filtered_df = artists_df[~artists_df['Wiki URL'].isna() & artists_df['Country'].isna()]
print("After fixing countries: ", len(filtered_df))

After fixing countries:  13


Well isnt that much btetter :)))))))))))))))))))))))

In [ ]:
artists_df.to_csv('artist_data_country_more.csv', index=False)

In [ ]:
artists_df = pd.read_csv("artist_data_country_more.csv")

In [ ]:
artists_df["Country"].unique()
# artists_df["Country"].value_counts(dropna=False)

array(['United States', ' U.S.', '(age\xa055)', nan, 'Canada',
       'United Kingdom', ' England', ' Canada', 'Sweden', ' Australia',
       ' United States', ' U.S.[1]', ' Los Angeles', 'Germany',
       ' North Carolina', 'Switzerland', ' U.S.[1][2]', 'Haiti',
       'Puerto Rico', 'Kingston', '(age\xa056)', ' Lithuania',
       ' Germany and United States', ' Puerto Rico', ' Sweden',
       'Australia', ' Netherlands', ' Texas[1]', ' U.S.[3]', 'Ireland',
       'Jamaica', 'Netherlands', ' United States[1]', ' Washington',
       '[2]', ' Canada[1]', 'South Korea', 'Spain', ' Ohio', ' U.S.[2]',
       ')', 'Garden of Eden', '(age\xa054)', '[1]', 'Norway', ' US',
       'Italy', 'England', 'U.S.', ' Belgium', 'New Zealand',
       ' the Bahamas', ' Ireland', ' France', 'Nigeria', ' Germany[1]',
       'Bolivia', '(age\xa044)', 'Brazil', ' Finland', ' Italy',
       'Belgium', ' California', 'Colombia', '(age\xa047)', 'Mexico',
       ' Spain', ' UK', ' Trinidad and Tobago', 'South Af

In [ ]:
us_vars=["U.S.", "USA","U.S","US","Virginia","North Carolina", "Michigan","New York City", "South Carolina","Los Angeles","Louisiana","Kentucky","Pennsylvania","Arizona",
         "Arkansas","Texas","Tennessee","Vermont","ArkansasUnited States","North Carolina","Pennsylvania","Washington","Ohio","California","Massachusetts",]

cb_vars=["Trinidad and Tobago","Puerto Rico","Saint Vincent and the Grenadines","Haiti", "The Bahamas"]

uk_vars = ["England", "U.K.", "UK", "London", "East Sussex", "Lincolnshire","Isle of Man","British Virgin Islands", "Wales", "Scotland", "Northern Ireland",
    "London", "East Sussex", "West Sussex", "Lincolnshire", "Manchester", "Birmingham",
    "Edinburgh", "Glasgow", "Cardiff", "Oxford", "Cambridge", "Staffordshire",
    "Hampshire", "Yorkshire", "Nottinghamshire", "Warwickshire", "Surrey", "Utah"]

none_vars=["St Kevins CollegeMonash University","Garden of Eden",""]


def clean_countries(text):
    if pd.isna(text):return None
    text=clean_parenthesis(text)
    return re.sub(r'[^a-zA-Z\s]', '', text)

In [ ]:
#truncate

artists_df["Country"] = artists_df["Country"].str.replace(r"\[.*?\]", "", regex=True)  # Remove the brackets []
artists_df["Country"] = artists_df["Country"].apply(lambda x: x.strip() if isinstance(x, str) else x)

artists_df["Country"] = artists_df["Country"].apply(lambda x: "United States" if isinstance(x, str) and x in us_vars else x)
artists_df["Country"] = artists_df["Country"].apply(lambda x: "United Kingdom" if isinstance(x, str) and x in uk_vars else x)
artists_df["Country"] = artists_df["Country"].apply(lambda x: "The Carrebian" if isinstance(x, str) and x in cb_vars else x)

artists_df.loc[artists_df['Country'] == "Israel", 'Country'] ="Palestine"
artists_df.loc[artists_df['Country'] == "Iraqi–Canadian", 'Country'] ="Iraq"
artists_df.loc[artists_df['Country'] == "United\xa0States", 'Country'] ="United States"
artists_df.loc[artists_df['Country'] == "West Germany", 'Country'] ="United States"
artists_df.loc[artists_df['Country'] == "Germany and United States", 'Country'] ="Germany"
artists_df.loc[artists_df['Country'] == "(present-day Kosovo)", 'Country'] ="Kosovo"
artists_df.loc[artists_df['Country'] == "SFR Yugoslavia(now Kosovo)", 'Country'] ="Kosovo"
artists_df.loc[artists_df['Country'] == "SFR Yugoslavia(now Kosovo)", 'Country'] ="Kosovo"
artists_df.loc[artists_df['Country'] == "Kingston", 'Country'] ="Jamaica"
artists_df["Country"] = artists_df["Country"].apply(lambda x: None if isinstance(x, str) and x in none_vars else x)
artists_df["Country"] = artists_df["Country"].apply(lambda x: clean_countries(x) if isinstance(x, str) else x)
artists_df["Country"] = artists_df["Country"].apply(lambda x: None if isinstance(x, str) and x in none_vars else x)
artists_df["Country"] = artists_df["Country"].apply(lambda x: None if pd.isna(x) else x)

artists_df['Country'].unique()


array(['United States', None, 'Canada', 'United Kingdom', 'Sweden',
       'Australia', 'Germany', 'Switzerland', 'The Carrebian', 'Jamaica',
       'Lithuania', 'Netherlands', 'Ireland', 'South Korea', 'Spain',
       'Norway', 'Italy', 'Belgium', 'New Zealand', 'the Bahamas',
       'France', 'Nigeria', 'Bolivia', 'Brazil', 'Finland', 'Colombia',
       'Mexico', 'South Africa', 'Russia', 'Barbados',
       'Dominican Republic', 'Greece', 'Lebanon', 'Iceland', 'Denmark',
       'Panama', 'Georgia', 'India', 'Philippines', 'Romania', 'Kosovo',
       'Vietnam', 'Guam', 'Japan', 'Cuba', 'China', 'Palestine', 'Iraq',
       'Guyana', 'Thailand', 'Ghana', 'Argentina', 'The Gambia',
       'Armenia', 'Uganda', 'Chile'], dtype=object)

In [ ]:
artists_df["Country"].value_counts(dropna=False)

,count
Country,
United States,1518
None,239
United Kingdom,147
Canada,68
Australia,31
The Carrebian,26
South Korea,20
Sweden,18
Mexico,18


greta, we just fixed so mnya countries. Now let us reclean all thta caus tbh its such a mess all over again.

In [ ]:
artists_df.to_csv('artist_data_country_final_fixed.csv', index=False)

In [ ]:
artists_df = pd.read_csv("artist_data_country_final_fixed.csv")

In [ ]:
artists_df #2253 rows × 11 columns

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2248,A11432,Chase Matthew,1,NaN,United States,1997-12-20,1.0,1,https://en.wikipedia.org/wiki/Chase_Matthew,M,Chase Matthew
2249,A11437,Tito Double P,1,NaN,Mexico,1997-08-18,1.0,1,https://en.wikipedia.org/wiki/Tito_Double_P,M,Tito Double P
2250,A11439,Addison Rae,1,Addison Rae Easterling,United States,2000-10-06,1.0,1,https://en.wikipedia.org/wiki/Addison_Rae,F,Addison Rae
2251,A11440,Surf Curse,1,NaN,United States,NaN,1.0,1,https://en.wikipedia.org/wiki/Surf_Curse,NaN,Surf Curse


## Merge with grammys data, add artists from grammys data


In [ ]:

grammys_df = pd.read_csv("grammys_data_cleaned_merge.csv") #840 x9

In [ ]:
grammys_df

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx
...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly


In [ ]:
import wikipediaapi
import requests
#return flag for found or not, and
def find_url_artists(artist_name):
  page = wiki_wiki.page(artist_name)
  print("try for",  artist_name)
  if page.exists():
      url= page.fullurl
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return 0, None #if the page was found but theires needs for clarification
          else: return 1, url #page found
  return 0, None #page not found

In [ ]:
urls=[]
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in grammys_df.iterrows():
  i+=1
  url= None
  artist_name=row["Main Artist"]
  if "(" in artist_name and ")" in artist_name: #get the full name of the artist which could be between arenthesis
    s= artist_name.find("(")
    e= artist_name.find(")")
    artist_name= artist_name[s+1:e]
  print("*****************")
  print(i, artist_name)
  found, url= find_url_artists(artist_name)
  #if the page wasnt found because on an mbiguity , try again with "singer"
  if found!=1:found, url= find_url_artists(artist_name+" (singer)")
  if found!=1:found, url= find_url_artists(artist_name+" (musician)")
  if found!=1:found, url= find_url_artists(artist_name+" (singer-songwriter)")
  if found!=1:found, url=find_url_artists(artist_name+" (rapper)")
  if found!=1:found, url= find_url_artists(artist_name+" (group)")
  if found!=1:found, url= find_url_artists(artist_name+" (band)")

  urls.append(url)

*****************
1 Kendrick Lamar
try for Kendrick Lamar
https://en.wikipedia.org/wiki/Kendrick_Lamar
*****************
2 Taylor Swift
try for Taylor Swift
https://en.wikipedia.org/wiki/Taylor_Swift
*****************
3 Chappell Roan
try for Chappell Roan
https://en.wikipedia.org/wiki/Chappell_Roan
*****************
4 Billie Eilish
try for Billie Eilish
https://en.wikipedia.org/wiki/Billie_Eilish
*****************
5 Charli Xcx
try for Charli Xcx
https://en.wikipedia.org/wiki/Charli_XCX
*****************
6 Sabrina Carpenter
try for Sabrina Carpenter
https://en.wikipedia.org/wiki/Sabrina_Carpenter
*****************
7 Beyoncé
try for Beyoncé
https://en.wikipedia.org/wiki/Beyonc%C3%A9
*****************
8 The Beatles
try for The Beatles
https://en.wikipedia.org/wiki/The_Beatles
*****************
9 Beyoncé
try for Beyoncé
https://en.wikipedia.org/wiki/Beyonc%C3%A9
*****************
10 Taylor Swift
try for Taylor Swift
https://en.wikipedia.org/wiki/Taylor_Swift
*****************
11 Chappell R

In [ ]:
print(len(urls))

840


In [ ]:
grammys_df["Wiki URL"]= urls

In [ ]:
grammys_df

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar,https://en.wikipedia.org/wiki/Kendrick_Lamar
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift,https://en.wikipedia.org/wiki/Taylor_Swift
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan,https://en.wikipedia.org/wiki/Chappell_Roan
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish,https://en.wikipedia.org/wiki/Billie_Eilish
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx,https://en.wikipedia.org/wiki/Charli_XCX
...,...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem,https://en.wikipedia.org/wiki/Eminem
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz,https://en.wikipedia.org/wiki/Jay-Z
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre,https://en.wikipedia.org/wiki/Dr._Dre
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly,https://en.wikipedia.org/wiki/Nelly


In [ ]:
grammys_df.to_csv('grammys_data_url.csv', index=False)

In [ ]:
grammys_df = pd.read_csv("grammys_data_url.csv")

In [ ]:
#match artists with their ids based on the url of wiki

import pandas as pd
import numpy as np

# Loop through each artist in artists_df
for idx, artist_row in artists_df.iterrows():
    # Get the artist's Wiki URL and ArtistID from artists_df
    artist_url = artist_row["Wiki URL"]
    artist_id = artist_row["ArtistID"]

    # Check if the Wiki URL is not None or NaN
    if pd.notna(artist_url):
        # Find all rows in grammys_df where the Wiki URL matches the artist's Wiki URL
        grammys_df.loc[grammys_df["Wiki URL"] == artist_url, "ArtistID"] = artist_id




In [ ]:
grammys_df #840 rows × 11 columns

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar,https://en.wikipedia.org/wiki/Kendrick_Lamar,A5155
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift,https://en.wikipedia.org/wiki/Taylor_Swift,A2657
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan,https://en.wikipedia.org/wiki/Chappell_Roan,A11167
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish,https://en.wikipedia.org/wiki/Billie_Eilish,A7541
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx,https://en.wikipedia.org/wiki/Charli_XCX,A5836
...,...,...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem,https://en.wikipedia.org/wiki/Eminem,A271
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz,https://en.wikipedia.org/wiki/Jay-Z,A107
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre,https://en.wikipedia.org/wiki/Dr._Dre,A517
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly,https://en.wikipedia.org/wiki/Nelly,A659


In [ ]:
temp= grammys_df[grammys_df['ArtistID'].isna()]
grammys_nan_artists= grammys_df[grammys_df['ArtistID'].isna() & ~grammys_df['Wiki URL'].isna()]
print("Nb of misisng artists", len(grammys_nan_artists))
print("Nb of NAN artists ids in total", len(temp))
grammys_nan_artists #70 rows × 11 columns

Nb of misisng artists 70
Nb of NAN artists ids in total 92


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
12,67,2024,Album Of The Year,Jacob Collier,Jacob Collier,Djesse Vol 4,Djesse Vol. 4,0,Jacob Collier,https://en.wikipedia.org/wiki/Jacob_Collier,NaN
20,67,2024,Best New Artist,Khruangbin,Khruangbin,NaN,NaN,0,Khruangbin,https://en.wikipedia.org/wiki/Khruangbin,NaN
36,67,2024,Best Rap Song,Rapsody,Rapsody Featuring Hit-Boy,Asteroids,Asteroids,0,Rapsody,https://en.wikipedia.org/wiki/Rapsody,NaN
48,66,2023,Record Of The Year,Boygenius,boygenius,Not Strong Enough,Not Strong Enough,0,Boygenius,https://en.wikipedia.org/wiki/Boygenius,NaN
49,66,2023,Record Of The Year,Jon Batiste,Jon Batiste,Worship,Worship,0,Jon Batiste,https://en.wikipedia.org/wiki/Jon_Batiste,NaN
...,...,...,...,...,...,...,...,...,...,...,...
752,46,2003,Song Of The Year,Warren Zevon,Warren Zevon,Keep Me In Your Heart,Keep Me In Your Heart,0,Warren Zevon,https://en.wikipedia.org/wiki/Warren_Zevon,NaN
798,44,2001,Album Of The Year,Bob Dylan,Bob Dylan,Love And Theft,Love And Theft,0,Bob Dylan,https://en.wikipedia.org/wiki/Bob_Dylan,NaN
820,43,2000,Album Of The Year,Steely Dan,Steely Dan,Two Against Nature,Two Against Nature,1,Steely Dan,https://en.wikipedia.org/wiki/Steely_Dan,NaN
821,43,2000,Album Of The Year,Paul Simon,Paul Simon,You'Re The One,You're The One,0,Paul Simon,https://en.wikipedia.org/wiki/Paul_Simon,NaN


## get missng info from other artists found in grammy csv

In [ ]:
import wikipediaapi
import requests


def find_artists_data(url):
  if url:
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return 0, None, None, None, None, None #if the page was found but theires needs for clarification

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")
          if group:
            group= group.find_next_sibling()
            group=group.find_all("li")

            member=len(group)
            name=None
            birthday=None
            nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
            if nationality:
              nationality= nationality.find_next_sibling()
              nationality= nationality.text.split(",")[-1] if nationality else None
          else: #not a group

            member=1
            #full name of the artist
            name= content.find("div", class_="nickname")
            name=name.text if name else None
            #brithday of the artist
            bday= content.find("span", class_="bday")
            birthday=bday.text if bday else None


             #birthplace of the artist
            nationality= content.find("div", class_="birthplace")
            if nationality:
              nationality= nationality.text.split(",")[-1]
            else:
              nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
              if nationality:
                nationality= nationality.find_next_sibling()
                nationality= nationality.text.split(",")[-1] if nationality else None
              else: nationality=None

            if nationality==None:
              born= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Born")
              if born:
                nationality= born.find_next_sibling() #next element thtc otnains the indo
                if nationality : nationality = nationality.find_all(recursive=False) #all children
                if len(nationality)==0: return None
                if nationality : nationality= nationality[-1] if nationality else None #last child
                if nationality: nationality = nationality.next_sibling.strip().split(",")[-1] if nationality.next_sibling else nationality.text.strip().split(",")[-1]
              else: nationality=None

      return 1, name, birthday, nationality, member, url

  else: return 0, None, None, None, None, None



In [ ]:
grammys_nan_artists = grammys_nan_artists.drop_duplicates(subset=['Wiki URL'])
print("new unqiue artists from grammy data", len(grammys_nan_artists))
grammys_nan_artists


new unqiue artists from grammy data 50


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
12,67,2024,Album Of The Year,Jacob Collier,Jacob Collier,Djesse Vol 4,Djesse Vol. 4,0,Jacob Collier,https://en.wikipedia.org/wiki/Jacob_Collier,NaN
20,67,2024,Best New Artist,Khruangbin,Khruangbin,NaN,NaN,0,Khruangbin,https://en.wikipedia.org/wiki/Khruangbin,NaN
36,67,2024,Best Rap Song,Rapsody,Rapsody Featuring Hit-Boy,Asteroids,Asteroids,0,Rapsody,https://en.wikipedia.org/wiki/Rapsody,NaN
48,66,2023,Record Of The Year,Boygenius,boygenius,Not Strong Enough,Not Strong Enough,0,Boygenius,https://en.wikipedia.org/wiki/Boygenius,NaN
49,66,2023,Record Of The Year,Jon Batiste,Jon Batiste,Worship,Worship,0,Jon Batiste,https://en.wikipedia.org/wiki/Jon_Batiste,NaN
64,66,2023,Best New Artist,Fred Again..,Fred again..,NaN,NaN,0,Fred Again,https://en.wikipedia.org/wiki/Fred_Again,NaN
93,65,2022,Record Of The Year,Abba,ABBA,Don'T Shut Me Down,Don't Shut Me Down,0,Abba,https://en.wikipedia.org/wiki/ABBA,NaN
104,65,2022,Best New Artist,Samara Joy,Samara Joy,NaN,NaN,1,Samara Joy,https://en.wikipedia.org/wiki/Samara_Joy,NaN
105,65,2022,Best New Artist,Wet Leg,Wet Leg,NaN,NaN,0,Wet Leg,https://en.wikipedia.org/wiki/Wet_Leg,NaN
106,65,2022,Best New Artist,Molly Tuttle,Molly Tuttle,NaN,NaN,0,Molly Tuttle,https://en.wikipedia.org/wiki/Molly_Tuttle,NaN


In [ ]:
i=0
found_wiki=[]
wiki_url=[]
birthdays=[]
spotify_founds=[]
nationalities=[]
names=[]
main_artists=[]
members=[]
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")


for idx, row in grammys_nan_artists.iterrows():
   found, name, bday, nationality, member, url= find_artists_data(row['Wiki URL'])
   if found==1:
    spotify_founds.append(0)
    main_artists.append(row['Main Artist'])
    found_wiki.append(found)
    names.append(name)
    birthdays.append(bday)
    nationalities.append(nationality)
    members.append(member)
    wiki_url.append(url)
   else: print (row)


https://en.wikipedia.org/wiki/Jacob_Collier
https://en.wikipedia.org/wiki/Khruangbin
https://en.wikipedia.org/wiki/Rapsody
https://en.wikipedia.org/wiki/Boygenius
https://en.wikipedia.org/wiki/Jon_Batiste
https://en.wikipedia.org/wiki/Fred_Again
https://en.wikipedia.org/wiki/ABBA
https://en.wikipedia.org/wiki/Samara_Joy
https://en.wikipedia.org/wiki/Wet_Leg
https://en.wikipedia.org/wiki/Molly_Tuttle
https://en.wikipedia.org/wiki/Tobe_Nwigwe
https://en.wikipedia.org/wiki/Domi_and_JD_Beck
https://en.wikipedia.org/wiki/Bonnie_Raitt
https://en.wikipedia.org/wiki/Arlo_Parks
https://en.wikipedia.org/wiki/Japanese_Breakfast
https://en.wikipedia.org/wiki/Finneas_O%27Connell
https://en.wikipedia.org/wiki/Arooj_Aftab
https://en.wikipedia.org/wiki/Black_Pumas
https://en.wikipedia.org/wiki/Haim_(band)
https://en.wikipedia.org/wiki/Kaytranada
https://en.wikipedia.org/wiki/D_Smoke
https://en.wikipedia.org/wiki/Chika_(rapper)
https://en.wikipedia.org/wiki/Phoebe_Bridgers
https://en.wikipedia.org/wiki

In [ ]:
print(len(found_wiki))#should be 50 for all
print(len(wiki_url))
print(len(names))
print(len(nationalities))
print(len(birthdays))
print(len(members))
print(len(main_artists))
print(len(spotify_founds))

50
50
50
50
50
50
50
50


In [ ]:
data={'Main Artist':main_artists, 'Spotify Found':spotify_founds, 	'Full Name':names,	'Country':nationalities,	'Birth Date':birthdays,	'Group Members':members,	'Found Wiki':found_wiki	,'Wiki URL':wiki_url}
temp_artist_df = pd.DataFrame(data)
temp_artist_df

,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL
0,Jacob Collier,0,None,England,1994-08-02,1,1,https://en.wikipedia.org/wiki/Jacob_Collier
1,Khruangbin,0,None,United States,None,3,1,https://en.wikipedia.org/wiki/Khruangbin
2,Rapsody,0,None,U.S.,1983-01-21,1,1,https://en.wikipedia.org/wiki/Rapsody
3,Boygenius,0,None,United States,None,3,1,https://en.wikipedia.org/wiki/Boygenius
4,Jon Batiste,0,None,,1986-11-11,1,1,https://en.wikipedia.org/wiki/Jon_Batiste
5,Fred Again..,0,None,England,1993-07-19,1,1,https://en.wikipedia.org/wiki/Fred_Again
6,Abba,0,None,Sweden,None,1,1,https://en.wikipedia.org/wiki/ABBA
7,Samara Joy,0,None,U.S.,1999-11-11,1,1,https://en.wikipedia.org/wiki/Samara_Joy
8,Wet Leg,0,None,England,None,0,1,https://en.wikipedia.org/wiki/Wet_Leg
9,Molly Tuttle,0,None,U.S.,1993-01-14,1,1,https://en.wikipedia.org/wiki/Molly_Tuttle


will proceed again to the cleaning of this data . First clean countries, then clean the names. then we merge it with the artists_df to have a complete list.

In [ ]:
temp_artist_df['Country'].unique()

array([' England', ' United States', ' U.S.', 'United States', '',
       ' Sweden', ' Pakistan', ' Canada', ' U.S.[1]', ' Australia'],
      dtype=object)

In [ ]:
temp_artist_df["Country"] = temp_artist_df["Country"].str.replace(r"\[.*?\]", "", regex=True)  # Remove the brackets []
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: x.strip() if isinstance(x, str) else x)

temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: "United States" if isinstance(x, str) and x in us_vars else x)
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: "United Kingdom" if isinstance(x, str) and x in uk_vars else x)
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: "The Carrebian" if isinstance(x, str) and x in cb_vars else x)

temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: None if isinstance(x, str) and x in none_vars else x)
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: clean_countries(x) if isinstance(x, str) else x)
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: None if isinstance(x, str) and x in none_vars else x)
temp_artist_df["Country"] = temp_artist_df["Country"].apply(lambda x: None if pd.isna(x) else x)

temp_artist_df['Country'].unique()


array(['United Kingdom', 'United States', None, 'Sweden', 'Pakistan',
       'Canada', 'Australia'], dtype=object)

In [ ]:
genders=[]
i=0
user_agent = "Dalas_projet/1.0 (random_mail@example.com)"
wiki_wiki = wikipediaapi.Wikipedia(user_agent=user_agent, language="en")
for idx, row in temp_artist_df.iterrows():
  i+=1
  print(i)
  gender=None
  if row["Found Wiki"]==1 and row["Wiki URL"] and row["Group Members"]==1 : #of course, we only look for 1 singer not for groups
    found, gender= find_gender(row["Wiki URL"])
    if gender==None: print("Issue with dteermining gender for\n", row)
  genders.append(gender)

1
2
3
4
5
6
7
Issue with dteermining gender for
 Main Artist                                    Abba
Spotify Found                                     0
Full Name                                      None
Country                                      Sweden
Birth Date                                     None
Group Members                                     1
Found Wiki                                        1
Wiki URL         https://en.wikipedia.org/wiki/ABBA
Name: 6, dtype: object
8
9
10
11
Issue with dteermining gender for
 Main Artist                                    Tobe Nwigwe
Spotify Found                                            0
Full Name                                             None
Country                                      United States
Birth Date                                      1987-03-08
Group Members                                            1
Found Wiki                                               1
Wiki URL         https://en.wikipedia.org/wiki/Tobe_Nw

In [ ]:
print(len(genders))#should be 50

50


In [ ]:
temp_artist_df["Gender"]=genders
temp_artist_df["Cleaned Main Artist"] = temp_artist_df["Main Artist"].astype(str).apply(lambda x: clean_artist_name(x))
temp_artist_df

,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,Jacob Collier,0,None,United Kingdom,1994-08-02,1,1,https://en.wikipedia.org/wiki/Jacob_Collier,M,Jacob Collier
1,Khruangbin,0,None,United States,None,3,1,https://en.wikipedia.org/wiki/Khruangbin,None,Khruangbin
2,Rapsody,0,None,United States,1983-01-21,1,1,https://en.wikipedia.org/wiki/Rapsody,F,Rapsody
3,Boygenius,0,None,United States,None,3,1,https://en.wikipedia.org/wiki/Boygenius,None,Boygenius
4,Jon Batiste,0,None,None,1986-11-11,1,1,https://en.wikipedia.org/wiki/Jon_Batiste,M,Jon Batiste
5,Fred Again..,0,None,United Kingdom,1993-07-19,1,1,https://en.wikipedia.org/wiki/Fred_Again,M,Fred Again
6,Abba,0,None,Sweden,None,1,1,https://en.wikipedia.org/wiki/ABBA,None,Abba
7,Samara Joy,0,None,United States,1999-11-11,1,1,https://en.wikipedia.org/wiki/Samara_Joy,F,Samara Joy
8,Wet Leg,0,None,United Kingdom,None,0,1,https://en.wikipedia.org/wiki/Wet_Leg,None,Wet Leg
9,Molly Tuttle,0,None,United States,1993-01-14,1,1,https://en.wikipedia.org/wiki/Molly_Tuttle,F,Molly Tuttle


okay perfect we added the artists from grammys to our artists dataset.
Now all w ehave to do is to tweak a bit that artist data and reod our indices to unique indices.

In [ ]:
artists_df.drop("ArtistID", axis=1, inplace=True)

In [ ]:
artists_df_combined = pd.concat([artists_df, temp_artist_df], ignore_index=True)

In [ ]:
artists_df_combined

,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...
2298,Warren Zevon,0,None,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,Steely Dan,0,None,United States,None,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,None,Steely Dan
2301,Paul Simon,0,None,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


In [ ]:
artists_df_combined.to_csv('artists_final_combined.csv', index=True)

In [ ]:

artists_df_combined = pd.read_csv("artists_final_combined.csv")
artists_df_combined

,Unnamed: 0,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan
2301,2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


In [ ]:
artists_df_combined.rename(columns={"Unnamed: 0": "ArtistID"}, inplace=True)

In [ ]:
artists_df_combined["ArtistID"] = artists_df_combined["ArtistID"].astype(str).apply(lambda x: "A" + x)

In [ ]:
artists_df_combined #2303x11


,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


In [ ]:
artists_df_combined.to_csv('artists_final_combined.csv', index=False)

## QUICK EXPLORATION OF FINAL ARTIST DATA

In [ ]:
artists_df = pd.read_csv("artists_final_combined.csv")

In [ ]:
artists_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2303 entries, 0 to 2302
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ArtistID             2303 non-null   object 
 1   Main Artist          2303 non-null   object 
 2   Spotify Found        2303 non-null   int64  
 3   Full Name            476 non-null    object 
 4   Country              2061 non-null   object 
 5   Birth Date           1511 non-null   object 
 6   Group Members        2131 non-null   float64
 7   Found Wiki           2303 non-null   int64  
 8   Wiki URL             2149 non-null   object 
 9   Gender               1430 non-null   object 
 10  Cleaned Main Artist  2302 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 198.0+ KB


In [ ]:
artists_df.apply(pd.Series.unique)

,0
ArtistID,"[A0, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, ..."
Main Artist,"[Aerosmith, Monica, Jennifer Paige, Usher, Bra..."
Spotify Found,"[1, 0]"
Full Name,"[nan, Monica Denise Arnold, Usher Raymond IV[1..."
Country,"[United States, nan, Canada, United Kingdom, S..."
Birth Date,"[nan, 1980-10-24, 1973-09-03, 1978-10-14, 1979..."
Group Members,"[5.0, 1.0, nan, 2.0, 3.0, 4.0, 6.0, 11.0, 7.0,..."
Found Wiki,"[1, 0]"
Wiki URL,"[https://en.wikipedia.org/wiki/Aerosmith, http..."
Gender,"[nan, F, M]"


In [ ]:
artists_df["Gender"].unique()
artists_df["Gender"].value_counts(dropna=False)

,count
Gender,
M,956
NaN,873
F,474


In [ ]:
grammys_df = pd.read_csv("grammys_data_url.csv")

In [ ]:
#match artists with their ids based on the url of wiki

import pandas as pd
import numpy as np

# Loop through each artist in artists_df
for idx, artist_row in artists_df.iterrows():
    # Get the artist's Wiki URL and ArtistID from artists_df
    artist_url = artist_row["Wiki URL"]
    artist_id = artist_row["ArtistID"]

    # Check if the Wiki URL is not None or NaN
    if pd.notna(artist_url):
        # Find all rows in grammys_df where the Wiki URL matches the artist's Wiki URL
        grammys_df.loc[grammys_df["Wiki URL"] == artist_url, "ArtistID"] = artist_id




In [ ]:
grammys_df #840 rows × 11 columns

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar,https://en.wikipedia.org/wiki/Kendrick_Lamar,A5155
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift,https://en.wikipedia.org/wiki/Taylor_Swift,A2657
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan,https://en.wikipedia.org/wiki/Chappell_Roan,A11167
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish,https://en.wikipedia.org/wiki/Billie_Eilish,A7541
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx,https://en.wikipedia.org/wiki/Charli_XCX,A5836
...,...,...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem,https://en.wikipedia.org/wiki/Eminem,A271
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz,https://en.wikipedia.org/wiki/Jay-Z,A107
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre,https://en.wikipedia.org/wiki/Dr._Dre,A517
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly,https://en.wikipedia.org/wiki/Nelly,A659


In [ ]:
artists_df["Found Wiki"].unique()
artists_df["Found Wiki"].value_counts(dropna=False)

,count
Found Wiki,
1,2149
0,154


In [ ]:
artists_df["Group Members"].unique()
artists_df["Group Members"].value_counts(dropna=False)

,count
Group Members,
1.0,1702
NaN,172
4.0,119
2.0,105
3.0,90
5.0,78
6.0,18
7.0,6
9.0,5


In [ ]:
print("Number of artists with missing gender: ",len(artists_df[artists_df["Gender"].isna()]))
print("Number of artists with missing gender because thye aint on wikipedia: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==0) ]))
print("Number of artists with missing gender because thye a group: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"]>1) ]))
print("Number of artists with missing gender because we couldnt find it: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"]==1) ]))
print("Number of artists with missing gender because thye a group and we couldnt retrive infos?: ",len(artists_df[(artists_df["Gender"].isna()) & (artists_df["Found Wiki"]==1) &(artists_df["Group Members"].isna()) ]))

Number of artists with missing gender:  873
Number of artists with missing gender because thye aint on wikipedia:  154
Number of artists with missing gender because thye a group:  428
Number of artists with missing gender because we couldnt find it:  272
Number of artists with missing gender because thye a group and we couldnt retrive infos?:  18


## ARTISTid IN GRMAMMY DATA

In [ ]:
grammys_df = pd.read_csv("grammys_data_url.csv")

In [ ]:
#match artists with their ids based on the url of wiki

import pandas as pd
import numpy as np

# Loop through each artist in artists_df
for idx, artist_row in artists_df.iterrows():
    # Get the artist's Wiki URL and ArtistID from artists_df
    artist_url = artist_row["Wiki URL"]
    artist_id = artist_row["ArtistID"]

    # Check if the Wiki URL is not None or NaN
    if pd.notna(artist_url):
        # Find all rows in grammys_df where the Wiki URL matches the artist's Wiki URL
        grammys_df.loc[grammys_df["Wiki URL"] == artist_url, "ArtistID"] = artist_id




In [ ]:
grammys_df #840 rows × 11 columns

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar,https://en.wikipedia.org/wiki/Kendrick_Lamar,A1382
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift,https://en.wikipedia.org/wiki/Taylor_Swift,A967
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan,https://en.wikipedia.org/wiki/Chappell_Roan,A2223
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish,https://en.wikipedia.org/wiki/Billie_Eilish,A1833
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx,https://en.wikipedia.org/wiki/Charli_XCX,A1524
...,...,...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem,https://en.wikipedia.org/wiki/Eminem,A220
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz,https://en.wikipedia.org/wiki/Jay-Z,A102
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre,https://en.wikipedia.org/wiki/Dr._Dre,A331
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly,https://en.wikipedia.org/wiki/Nelly,A381


In [ ]:
temp= grammys_df[grammys_df['ArtistID'].isna()]
grammys_nan_artists= grammys_df[grammys_df['ArtistID'].isna() & ~grammys_df['Wiki URL'].isna()]
print("Nb of misisng artists when wiki is found", len(grammys_nan_artists))
print("Nb of NAN artists ids in total", len(temp))
temp #70 rows × 11 columns

Nb of misisng artists when wiki is found 0
Nb of NAN artists ids in total 22


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
59,66,2023,Best New Artist,The War,The War And Treaty,NaN,NaN,0,The War,NaN,NaN
208,63,2020,Song Of The Year,Jp Saxe,JP Saxe Featuring Julia Michaels,If The World Was Ending,If The World Was Ending,0,Jp Saxe,NaN,NaN
221,63,2020,Best Rap Album,"Royce 5’9""","Royce 5’9""",The Allegory,The Allegory,0,Royce 59,NaN,NaN
229,62,2019,Record Of The Year,Khalid,Khalid,Talk,Talk,0,Khalid,NaN,NaN
261,62,2019,Best Rap Song,Ybn Cordae,YBN Cordae Featuring Chance The Rapper,Bad Idea,Bad Idea,0,Ybn Cordae,NaN,NaN
263,62,2019,Best Rap Album,Ybn Cordae,YBN Cordae,The Lost Boy,The Lost Boy,0,Ybn Cordae,NaN,NaN
266,62,2019,Best Rap Album,Dreamville,Dreamville,Revenge Of The Dreamers Iii,Revenge Of The Dreamers III,0,Dreamville,NaN,NaN
276,61,2018,Album Of The Year,(Various Artists),(Various Artists),Black Panther: The Album Music From And Inspir...,"Black Panther: The Album, Music From And Inspi...",0,NaN,NaN,NaN
290,61,2018,Best New Artist,Chloe,Chloe x Halle,NaN,NaN,0,Chloe,NaN,NaN
323,60,2017,Best New Artist,Khalid,Khalid,NaN,NaN,0,Khalid,NaN,NaN


pour les quelques cas qui restent, on adjuste manuellement


In [ ]:
artists_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


In [ ]:
for idx, grammy_artist_row in temp.iterrows():
  grammy_artist_name = grammy_artist_row["Main Artist"]

  # Get the artist's ID from artists_df
  artist_id = artists_df.loc[artists_df["Main Artist"] == grammy_artist_name, "ArtistID"]

  # Ensure artist_id is not empty before assignment
  if not artist_id.empty:
      grammys_df.loc[grammys_df["Main Artist"] == grammy_artist_name, "ArtistID"] = artist_id.iloc[0]



In [ ]:
temp

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
59,66,2023,Best New Artist,The War,The War And Treaty,NaN,NaN,0,The War,NaN,NaN
208,63,2020,Song Of The Year,Jp Saxe,JP Saxe Featuring Julia Michaels,If The World Was Ending,If The World Was Ending,0,Jp Saxe,NaN,NaN
221,63,2020,Best Rap Album,"Royce 5’9""","Royce 5’9""",The Allegory,The Allegory,0,Royce 59,NaN,NaN
229,62,2019,Record Of The Year,Khalid,Khalid,Talk,Talk,0,Khalid,NaN,A1736
261,62,2019,Best Rap Song,Ybn Cordae,YBN Cordae Featuring Chance The Rapper,Bad Idea,Bad Idea,0,Ybn Cordae,NaN,NaN
263,62,2019,Best Rap Album,Ybn Cordae,YBN Cordae,The Lost Boy,The Lost Boy,0,Ybn Cordae,NaN,NaN
266,62,2019,Best Rap Album,Dreamville,Dreamville,Revenge Of The Dreamers Iii,Revenge Of The Dreamers III,0,Dreamville,NaN,A1902
276,61,2018,Album Of The Year,(Various Artists),(Various Artists),Black Panther: The Album Music From And Inspir...,"Black Panther: The Album, Music From And Inspi...",0,NaN,NaN,NaN
290,61,2018,Best New Artist,Chloe,Chloe x Halle,NaN,NaN,0,Chloe,NaN,A1961
323,60,2017,Best New Artist,Khalid,Khalid,NaN,NaN,0,Khalid,NaN,A1736


In [ ]:
temp= grammys_df[grammys_df['ArtistID'].isna()]

print("Nb of NAN artists ids in total", len(temp))
temp #70 rows × 11 columns

Nb of NAN artists ids in total 12


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
59,66,2023,Best New Artist,The War,The War And Treaty,NaN,NaN,0,The War,NaN,NaN
208,63,2020,Song Of The Year,Jp Saxe,JP Saxe Featuring Julia Michaels,If The World Was Ending,If The World Was Ending,0,Jp Saxe,NaN,NaN
221,63,2020,Best Rap Album,"Royce 5’9""","Royce 5’9""",The Allegory,The Allegory,0,Royce 59,NaN,NaN
261,62,2019,Best Rap Song,Ybn Cordae,YBN Cordae Featuring Chance The Rapper,Bad Idea,Bad Idea,0,Ybn Cordae,NaN,NaN
263,62,2019,Best Rap Album,Ybn Cordae,YBN Cordae,The Lost Boy,The Lost Boy,0,Ybn Cordae,NaN,NaN
276,61,2018,Album Of The Year,(Various Artists),(Various Artists),Black Panther: The Album Music From And Inspir...,"Black Panther: The Album, Music From And Inspi...",0,NaN,NaN,NaN
586,51,2008,Record Of The Year,M.I.A,M.I.A,Paper Planes,Paper Planes,0,Mia,NaN,NaN
633,50,2007,Song Of The Year,Plain White T'S,Plain White T's,Hey There Delilah,Hey There Delilah,0,Plain White Ts,NaN,NaN
732,47,2004,Best Rap Album,Ll Cool J,LL Cool J,The Definition,The Definition,0,Ll Cool J,NaN,NaN
795,44,2001,Album Of The Year,(Various Artists),(Various Artists),O Brother Where Art Thou? - Soundtrack,"O Brother, Where Art Thou? - Soundtrack",1,NaN,NaN,NaN


In [ ]:
for idx, grammy_artist_row in temp.iterrows():
  grammy_artist_name = grammy_artist_row["Main Artist"]

  # Get the artist's ID from artists_df
  artist_id = artists_df.loc[artists_df["Cleaned Main Artist"] == grammy_artist_name, "ArtistID"]

  # Ensure artist_id is not empty before assignment
  if not artist_id.empty:
      grammys_df.loc[grammys_df["Cleaned Main Artist"] == grammy_artist_name, "ArtistID"] = artist_id.iloc[0]



In [ ]:
temp= grammys_df[grammys_df['ArtistID'].isna()]

print("Nb of NAN artists ids in total", len(temp))
temp #70 rows × 11 columns

Nb of NAN artists ids in total 10


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
59,66,2023,Best New Artist,The War,The War And Treaty,NaN,NaN,0,The War,NaN,NaN
221,63,2020,Best Rap Album,"Royce 5’9""","Royce 5’9""",The Allegory,The Allegory,0,Royce 59,NaN,NaN
261,62,2019,Best Rap Song,Ybn Cordae,YBN Cordae Featuring Chance The Rapper,Bad Idea,Bad Idea,0,Ybn Cordae,NaN,NaN
263,62,2019,Best Rap Album,Ybn Cordae,YBN Cordae,The Lost Boy,The Lost Boy,0,Ybn Cordae,NaN,NaN
276,61,2018,Album Of The Year,(Various Artists),(Various Artists),Black Panther: The Album Music From And Inspir...,"Black Panther: The Album, Music From And Inspi...",0,NaN,NaN,NaN
586,51,2008,Record Of The Year,M.I.A,M.I.A,Paper Planes,Paper Planes,0,Mia,NaN,NaN
633,50,2007,Song Of The Year,Plain White T'S,Plain White T's,Hey There Delilah,Hey There Delilah,0,Plain White Ts,NaN,NaN
795,44,2001,Album Of The Year,(Various Artists),(Various Artists),O Brother Where Art Thou? - Soundtrack,"O Brother, Where Art Thou? - Soundtrack",1,NaN,NaN,NaN
816,43,2000,Record Of The Year,Destiny'S Child,Destiny's Child,Say My Name,Say My Name,0,Destinys Child,NaN,NaN
831,43,2000,Song Of The Year,Destiny'S Child,Destiny's Child,Say My Name,Say My Name,0,Destinys Child,NaN,NaN


In [ ]:
for idx, grammy_artist_row in temp.iterrows():
  grammy_artist_name = grammy_artist_row["Cleaned Main Artist"]

  # Get the artist's ID from artists_df
  artist_id = artists_df.loc[artists_df["Cleaned Main Artist"] == grammy_artist_name, "ArtistID"]

  # Ensure artist_id is not empty before assignment
  if not artist_id.empty:
      grammys_df.loc[grammys_df["Cleaned Main Artist"] == grammy_artist_name, "ArtistID"] = artist_id.iloc[0]

In [ ]:
temp= grammys_df[grammys_df['ArtistID'].isna()]

print("Nb of NAN artists ids in total", len(temp))
temp #70 rows × 11 columns

Nb of NAN artists ids in total 6


,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
59,66,2023,Best New Artist,The War,The War And Treaty,NaN,NaN,0,The War,NaN,NaN
221,63,2020,Best Rap Album,"Royce 5’9""","Royce 5’9""",The Allegory,The Allegory,0,Royce 59,NaN,NaN
261,62,2019,Best Rap Song,Ybn Cordae,YBN Cordae Featuring Chance The Rapper,Bad Idea,Bad Idea,0,Ybn Cordae,NaN,NaN
263,62,2019,Best Rap Album,Ybn Cordae,YBN Cordae,The Lost Boy,The Lost Boy,0,Ybn Cordae,NaN,NaN
276,61,2018,Album Of The Year,(Various Artists),(Various Artists),Black Panther: The Album Music From And Inspir...,"Black Panther: The Album, Music From And Inspi...",0,NaN,NaN,NaN
795,44,2001,Album Of The Year,(Various Artists),(Various Artists),O Brother Where Art Thou? - Soundtrack,"O Brother, Where Art Thou? - Soundtrack",1,NaN,NaN,NaN


In [ ]:
grammys_df

,Grammy,Year,Category,Main Artist,Artist,Cleaned Song or Album,Song or Album,Win,Cleaned Main Artist,Wiki URL,ArtistID
0,67,2024,Record Of The Year,Kendrick Lamar,Kendrick Lamar,Not Like Us,Not Like Us,1,Kendrick Lamar,https://en.wikipedia.org/wiki/Kendrick_Lamar,A1382
1,67,2024,Record Of The Year,Taylor Swift,Taylor Swift Featuring Post Malone,Fortnight,Fortnight,0,Taylor Swift,https://en.wikipedia.org/wiki/Taylor_Swift,A967
2,67,2024,Record Of The Year,Chappell Roan,Chappell Roan,Good Luck Babe!,"Good Luck, Babe!",0,Chappell Roan,https://en.wikipedia.org/wiki/Chappell_Roan,A2223
3,67,2024,Record Of The Year,Billie Eilish,Billie Eilish,Birds Of A Feather,BIRDS OF A FEATHER,0,Billie Eilish,https://en.wikipedia.org/wiki/Billie_Eilish,A1833
4,67,2024,Record Of The Year,Charli Xcx,Charli xcx,360,360,0,Charli Xcx,https://en.wikipedia.org/wiki/Charli_XCX,A1524
...,...,...,...,...,...,...,...,...,...,...,...
835,43,2000,Best Rap Album,Eminem,Eminem,The Marshall Mathers Lp,The Marshall Mathers LP,1,Eminem,https://en.wikipedia.org/wiki/Eminem,A220
836,43,2000,Best Rap Album,Jay-Z,Jay-Z,Vol 3. Life And Times Of S Carter,Vol. 3...Life And Times Of S. Carter,0,Jayz,https://en.wikipedia.org/wiki/Jay-Z,A102
837,43,2000,Best Rap Album,Dr. Dre,Dr. Dre,Dr Dre - 2001,Dr. Dre - 2001,0,Dr Dre,https://en.wikipedia.org/wiki/Dr._Dre,A331
838,43,2000,Best Rap Album,Nelly,Nelly,Country Grammar,Country Grammar,0,Nelly,https://en.wikipedia.org/wiki/Nelly,A381


In [ ]:
grammys_df.loc[grammys_df["Main Artist"]=="M.I.A", "ArtistID"]='A1112'
grammys_df.loc[grammys_df["Main Artist"]=="M.I.A", "Main Artist"]="M.I.A."

In [ ]:
grammys_df.to_csv('grammys_merged_with_artit_final.csv', index=False)

# US STATE

So on a eu la petite idee de cheker les states caus une grande majorite des artistses vienennt des US. donc on va verifier tout ca et trouver par state ensuite on va fair eune analyse par state

In [ ]:
#scrap data about states
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

url = "https://50statesofmusic.com/"
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raise an error if the request fails

# Parse the HTML content
soup = BeautifulSoup(response.text, "html.parser")
states=[]
urls=[]

#get all states
stat_container= soup.find("div", class_=['expanded-navbar']).find("div", class_=["container"])
# print(stat_container)
lis= stat_container.find_all("li")
for li in lis:
  a_tag = li.find("a")
  if a_tag:
      href = a_tag.get("href")
      state_name = a_tag.get_text(strip=True)
  else:
    href=None
    state_name=None
  states.append(state_name)
  urls.append(href)

states_df = pd.DataFrame({"State": states, "URL": urls})



In [ ]:
states_df= states_df[:-1]

In [ ]:
budgets=[]
estabs=[]
jobs=[]
events=[]
schools=[]

for idx, row in states_df.iterrows():
  url= row['URL']
  print(url)
  response = requests.get(url, headers=headers)
  response.raise_for_status()  # Raise an error if the request fails
  soup = BeautifulSoup(response.text, "html.parser")
  #init
  budget, estab, event, school, job= None, None, None, None, None

  #search
  container= soup.find("div", class_=['home-numbers']).find("div", class_=['container'])
  budget= container.find("div", class_=['numbers-lead-text']).find('h3', class_=['blue']).text #budget
  sub_container= container.find("div",class_=['numbers-sub-text'])
  data= sub_container.find_all("h3", class_=['blue'])
  job=data[0].text #nb of jobs
  estab= data[1].text #nb of establishments


  container= soup.find("div", class_=["events"])
  events_cont= container.find_all("h4")
  for event_h4 in events_cont:
    if event_h4.text=="Music Venues & Festivals":
      event_data= event_h4.find_next().find_all("li")
      event=len(event_data) if event_data else None #find nb of festivals and all in the state


    elif event_h4.text=="Music Schools":
      schools_data= event_h4.find_next().find_all("li")
      school=len(schools_data) if schools_data else None #find nb of festivals and all in the state

  print(budget, job, estab, event, school)
  print('*'*100)
  #append
  budgets.append(budget)
  jobs.append(job)
  estabs.append(estab)
  events.append(event)
  schools.append(school)


https://50statesofmusic.com/state/alabama/
$692.5M 13,028 778 34 2
****************************************************************************************************
https://50statesofmusic.com/state/alaska/
$190.3M 3,650 476 15 2
****************************************************************************************************
https://50statesofmusic.com/state/arizona/
$1.9B 30,934 4,424 57 2
****************************************************************************************************
https://50statesofmusic.com/state/arkansas/
$342.2M 6,158 599 23 1
****************************************************************************************************
https://50statesofmusic.com/state/california/
$51.3B 460,127 80,433 255 14
****************************************************************************************************
https://50statesofmusic.com/state/colorado/
$2.8B 50,146 8,221 82 5
**************************************************************************************

In [ ]:
states_df["Budget"]= budgets
states_df["Jobs"]=jobs
states_df["Music Establishments"]=estabs
states_df["Events and Festivals"]=events
states_df["Music Instituitions"]=schools
states_df

,State,URL,Budget,Jobs,Music Establishments,Events and Festivals,Music Instituitions
0,Alabama,https://50statesofmusic.com/state/alabama/,$692.5M,"13,028",778,34,2.0
1,Alaska,https://50statesofmusic.com/state/alaska/,$190.3M,"3,650",476,15,2.0
2,Arizona,https://50statesofmusic.com/state/arizona/,$1.9B,"30,934","4,424",57,2.0
3,Arkansas,https://50statesofmusic.com/state/arkansas/,$342.2M,"6,158",599,23,1.0
4,California,https://50statesofmusic.com/state/california/,$51.3B,"460,127","80,433",255,14.0
5,Colorado,https://50statesofmusic.com/state/colorado/,$2.8B,"50,146","8,221",82,5.0
6,Connecticut,https://50statesofmusic.com/state/connecticut/,$1.3B,"15,195","1,257",37,4.0
7,Delaware,https://50statesofmusic.com/state/delaware/,$204.3M,"3,204","1,144",15,1.0
8,District of Columbia,https://50statesofmusic.com/state/district-of-...,$1.1B,"4,811",266,28,2.0
9,Florida,https://50statesofmusic.com/state/florida/,$9.3B,"169,706","22,533",84,15.0


In [ ]:
def clean_budget(value): #convert my billions and milliosn to real numbers
  if 'M' in value:
      return float(value.replace('$', '').replace('M', '').strip()) * 1e6  # Millions
  elif 'B' in value:
      return float(value.replace('$', '').replace('B', '').strip()) * 1e9  # Billions
  elif 'K' in value:
      return float(value.replace('$', '').replace('K', '').strip()) * 1e3  # Thousands
  return float(value.replace('$', '').strip())  # Handle if no suffix
  return value

In [ ]:
# Replace NaN values with 0
states_df['Music Instituitions'] = states_df['Music Instituitions'].fillna(0)
states_df['Clean Budget']=states_df['Budget'].apply(lambda x: clean_budget(x) if x else None)
states_df

,State,URL,Budget,Jobs,Music Establishments,Events and Festivals,Music Instituitions,Clean Budget
0,Alabama,https://50statesofmusic.com/state/alabama/,$692.5M,"13,028",778,34,2.0,6.925000e+08
1,Alaska,https://50statesofmusic.com/state/alaska/,$190.3M,"3,650",476,15,2.0,1.903000e+08
2,Arizona,https://50statesofmusic.com/state/arizona/,$1.9B,"30,934","4,424",57,2.0,1.900000e+09
3,Arkansas,https://50statesofmusic.com/state/arkansas/,$342.2M,"6,158",599,23,1.0,3.422000e+08
4,California,https://50statesofmusic.com/state/california/,$51.3B,"460,127","80,433",255,14.0,5.130000e+10
5,Colorado,https://50statesofmusic.com/state/colorado/,$2.8B,"50,146","8,221",82,5.0,2.800000e+09
6,Connecticut,https://50statesofmusic.com/state/connecticut/,$1.3B,"15,195","1,257",37,4.0,1.300000e+09
7,Delaware,https://50statesofmusic.com/state/delaware/,$204.3M,"3,204","1,144",15,1.0,2.043000e+08
8,District of Columbia,https://50statesofmusic.com/state/district-of-...,$1.1B,"4,811",266,28,2.0,1.100000e+09
9,Florida,https://50statesofmusic.com/state/florida/,$9.3B,"169,706","22,533",84,15.0,9.300000e+09


In [ ]:
states_df.to_csv("states_data.csv", index=True)

In [ ]:
states_df= pd.read_csv("states_data.csv")
states_df

,Unnamed: 0,State,URL,Budget,Jobs,Music Establishments,Events and Festivals,Music Instituitions,Clean Budget
0,0,Alabama,https://50statesofmusic.com/state/alabama/,$692.5M,"13,028",778,34,2.0,6.925000e+08
1,1,Alaska,https://50statesofmusic.com/state/alaska/,$190.3M,"3,650",476,15,2.0,1.903000e+08
2,2,Arizona,https://50statesofmusic.com/state/arizona/,$1.9B,"30,934","4,424",57,2.0,1.900000e+09
3,3,Arkansas,https://50statesofmusic.com/state/arkansas/,$342.2M,"6,158",599,23,1.0,3.422000e+08
4,4,California,https://50statesofmusic.com/state/california/,$51.3B,"460,127","80,433",255,14.0,5.130000e+10
5,5,Colorado,https://50statesofmusic.com/state/colorado/,$2.8B,"50,146","8,221",82,5.0,2.800000e+09
6,6,Connecticut,https://50statesofmusic.com/state/connecticut/,$1.3B,"15,195","1,257",37,4.0,1.300000e+09
7,7,Delaware,https://50statesofmusic.com/state/delaware/,$204.3M,"3,204","1,144",15,1.0,2.043000e+08
8,8,District of Columbia,https://50statesofmusic.com/state/district-of-...,$1.1B,"4,811",266,28,2.0,1.100000e+09
9,9,Florida,https://50statesofmusic.com/state/florida/,$9.3B,"169,706","22,533",84,15.0,9.300000e+09


In [ ]:
states_df.rename(columns={'Unnamed: 0': 'State ID'}, inplace=True)

In [ ]:
states_df["State ID"]= states_df["State ID"].apply(lambda x: "us"+str(x))
states_df

,State ID,State,URL,Budget,Jobs,Music Establishments,Events and Festivals,Music Instituitions,Clean Budget
0,us0,Alabama,https://50statesofmusic.com/state/alabama/,$692.5M,"13,028",778,34,2.0,6.925000e+08
1,us1,Alaska,https://50statesofmusic.com/state/alaska/,$190.3M,"3,650",476,15,2.0,1.903000e+08
2,us2,Arizona,https://50statesofmusic.com/state/arizona/,$1.9B,"30,934","4,424",57,2.0,1.900000e+09
3,us3,Arkansas,https://50statesofmusic.com/state/arkansas/,$342.2M,"6,158",599,23,1.0,3.422000e+08
4,us4,California,https://50statesofmusic.com/state/california/,$51.3B,"460,127","80,433",255,14.0,5.130000e+10
5,us5,Colorado,https://50statesofmusic.com/state/colorado/,$2.8B,"50,146","8,221",82,5.0,2.800000e+09
6,us6,Connecticut,https://50statesofmusic.com/state/connecticut/,$1.3B,"15,195","1,257",37,4.0,1.300000e+09
7,us7,Delaware,https://50statesofmusic.com/state/delaware/,$204.3M,"3,204","1,144",15,1.0,2.043000e+08
8,us8,District of Columbia,https://50statesofmusic.com/state/district-of-...,$1.1B,"4,811",266,28,2.0,1.100000e+09
9,us9,Florida,https://50statesofmusic.com/state/florida/,$9.3B,"169,706","22,533",84,15.0,9.300000e+09


In [ ]:
#fix types first
states_df.dtypes

,0
State ID,object
State,object
URL,object
Budget,object
Jobs,object
Music Establishments,object
Events and Festivals,int64
Music Instituitions,float64
Clean Budget,float64


In [ ]:
states_df["Music Establishments"]= states_df["Music Establishments"].apply(lambda x: int(x.replace(",", "")))
states_df["Jobs"]= states_df["Jobs"].apply(lambda x: int(x.replace(",", "")))
states_df.dtypes

,0
State ID,object
State,object
URL,object
Budget,object
Jobs,int64
Music Establishments,int64
Events and Festivals,int64
Music Instituitions,float64
Clean Budget,float64


In [ ]:
states_df.to_csv("states_data.csv", index=False)

## US state of US artists
ok now we have the infos for individual states , we'll get where each states belongs

In [ ]:
import pandas as pd

artists_df = pd.read_csv("artists_final_combined.csv")
artists_df #2303 x 11

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.
Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
artists_US_df= artists_df[artists_df['Country']=="United States" ]
artists_US_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy
...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon


In [ ]:
import wikipediaapi
import requests


def find_artists_state(url):
  if url:
      print(url)
      response = requests.get(url)
      if response.status_code == 200:
          soup = BeautifulSoup(response.text, "html.parser")
          content = soup.find("table", class_=["infobox vcard plainlist","infobox biography vcard","infobox ib-youtube biography vcard"])
          # print(content)
          if content==None: return None

          #is it a group or a single artist
          group= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Members")
          if group:
            nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")

            if nationality:
              nationality= nationality.find_next_sibling()
              nationality=nationality.get_text(strip=True)
              print(nationality)
              nationality= nationality.split(",")
              print(nationality)
              if len(nationality)>2: nationality= nationality[-2]
              if len(nationality)==2 and (nationality[-1].strip()=="U.S." or nationality[-1].strip()=="US"): nationality= nationality[-2]
            else : nationality=None
          else: #not a group

            #birthplace of the artist
            nationality= content.find("div", class_="birthplace")
            if nationality:
              nationality= nationality.text.split(",")[-2]
            else:
              nationality= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Origin")
              if nationality:
                nationality= nationality.find_next_sibling()
                nationality=nationality.get_text(strip=True)
                print(nationality)
                nationality= nationality.split(",")
                print(nationality)
                if len(nationality)>2: nationality= nationality[-2]
                if len(nationality)==2 and (nationality[-1].strip()=="U.S." or nationality[-1].strip()=="US"): nationality= nationality[-2]
              else: nationality=None

            if nationality==None:
              born= content.find("th", {"class": "infobox-label"}, string=lambda text: text and text.strip() == "Born")
              if born:
                nationality= born.find_next_sibling() #next element thtc otnains the indo
                nationality=nationality.get_text(strip=True)
                print(nationality)
                nationality= nationality.split(",")
                print(nationality)
                if len(nationality)>2: nationality= nationality[-2]
                if len(nationality)==2 and (nationality[-1].strip()=="U.S." or nationality[-1].strip()=="US"): nationality= nationality[-2]
              else: nationality=None
      if nationality!= None: return nationality

  return None



In [ ]:
us_states=[]

for idx, row in artists_df.iterrows():

  state=None
  if row['Country']=="United States":
    state= find_artists_state(row['Wiki URL'])
    print('*'*10 ,state) #coulnt find the state in the US
  us_states.append(state)

Streaming output truncated to the last 5000 lines.
['Philadelphia', 'Pennsylvania', ' U.S.']
********** Pennsylvania
https://en.wikipedia.org/wiki/Billy_Ray_Cyrus
**********  Kentucky
https://en.wikipedia.org/wiki/Trisha_Yearwood
**********  Georgia
https://en.wikipedia.org/wiki/The_Kinleys
Philadelphia, Pennsylvania, U.S.
['Philadelphia', ' Pennsylvania', ' U.S.']
**********  Pennsylvania
https://en.wikipedia.org/wiki/Cool_Breeze_(rapper)
East Point, Georgia, U.S.
['East Point', ' Georgia', ' U.S.']
**********  Georgia
https://en.wikipedia.org/wiki/Tevin_Campbell
Dallas, Texas, U.S.
['Dallas', ' Texas', ' U.S.']
**********  Texas
https://en.wikipedia.org/wiki/Jennifer_Love_Hewitt
**********  Texas
https://en.wikipedia.org/wiki/Mariah_Carey
**********  New York
https://en.wikipedia.org/wiki/Juvenile_(rapper)
(1975-03-26)March 26, 1975(age 50)New Orleans,Louisiana, U.S.
['(1975-03-26)March 26', ' 1975(age\xa050)New Orleans', 'Louisiana', ' U.S.']
********** Louisiana
https://en.wikipedi

In [ ]:
print(len(us_states))

2303


In [ ]:
artists_df["US State"]=us_states

In [ ]:
artists_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist,US State
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith,Massachusetts
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica,Georgia
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige,Georgia
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher,Texas
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon,Illinois
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan,Minnesota
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan,New York
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon,New York City


In [ ]:
artists_df[artists_df["Country"]=="United States"]['US State'].value_counts()


,count
US State,
California,181
Tennessee,93
Texas,91
Georgia,89
New York City,77
...,...
U.S.North Texas,1
"[Apple Valley, California]",1
1999(age 25)Kalama,1


In [ ]:
import re
def clean_us_state(text):
  text= re.sub(r"\[.*?\]", "", text)
  text= re.sub(r"\(.*?\)", "", text)
  text=re.sub(r'\d+', '', text)
  return text.strip()



In [ ]:
count=artists_df['US State'].apply(lambda x: isinstance(x, list)).sum()
count

np.int64(53)

In [ ]:
artists_df.loc[:,"US State"] = artists_df["US State"].apply(lambda x: x[-1] if isinstance(x, list) else x)
artists_df.loc[:,"US State"] = artists_df["US State"].apply(lambda x: clean_us_state(x) if pd.notna(x) else x)

In [ ]:
artists_df["US State"].value_counts()

,count
US State,
California,255
Georgia,130
Texas,118
New York City,116
Tennessee,103
...,...
San Jose,1
Toronto,1
Paradise,1


In [ ]:
artists_df["US State"].unique()

array(['Massachusetts', 'Georgia', 'Texas', 'Mississippi', 'Wisconsin',
       None, 'New York', 'New York City', 'Pennsylvania', 'California',
       'Maryland', 'United States', 'North Carolina', 'New Jersey',
       'Louisiana', 'D.C.', 'Michigan', 'Indiana', 'Florida',
       'South Central', 'Oklahoma', 'Illinois', 'Tennessee', 'Washington',
       'Kentucky', 'Arizona', 'Maine', 'Missouri', 'Utah', 'Virginia',
       'Oregon', 'Alabama', 'Minnesota', 'Nevada', 'Nebraska', 'Ohio',
       'West Virginia', 'Iowa', 'New Hampshire', 'Kansas',
       'South Carolina', 'Hawaii', 'U.S.', 'NY', 'Rhode Island',
       'Santa Barbara', 'Colorado', 'U.S.Palmdale', 'Arkansas',
       'MassachusettsU.S.', 'Connecticut', 'West Germany', 'Vermont',
       'New Mexico', 'Delaware', 'Los Angeles', 'St. Louis',
       'North Dakota', 'Decatur', 'ArkansasUnited States', 'Waitsfield',
       'Boston', 'Reseda', 'Knoxville', 'Alaska', 'Union City', 'Hawaiʻi',
       'Shreveport', 'Wyoming', 'Nashville

In [ ]:
ny_vars = ["New York", "New York City", "NY"]
ak_vars=["ArkansasUnited States"]
ca_vars = ["California", "Los Angeles", "Santa Barbara", "San Jose", "Reseda", "Paradise", "South Central","U.S.Palmdale","Union City"]
tx_vars = ["Texas", "U.S.North Texas", "North Texas"]
ma_vars = ["Massachusetts", "Boston", "MassachusettsU.S."]
ar_vars = ["Arkansas", "ArkansasUnited States"]
la_vars = ["Louisiana", "Shreveport"]
tn_vars = ["Tennessee", "Nashville", "Knoxville"]
mo_vars = ["Missouri", "St. Louis"]
vt_vars = ["Vermont", "Waitsfield"]
ga_vars = ["Georgia", "Decatur"]
hw_vars = ["Hawaiʻi"]
nj_vars = ["New Jersey", "Union City"]
wa_vars = ["Washington", "Kalama"]
null_vars=["Toronto","United States","US","West Germany"]

artists_df.loc[artists_df['US State'].isin(ny_vars), 'US State'] = "New York City"
artists_df.loc[artists_df['US State'].isin(ak_vars), 'US State'] = "Arkansas"
artists_df.loc[artists_df['US State'].isin(ca_vars), 'US State'] = "California"
artists_df.loc[artists_df['US State'].isin(tx_vars), 'US State'] = "Texas"
artists_df.loc[artists_df['US State'].isin(ma_vars), 'US State'] = "Massachusetts"
artists_df.loc[artists_df['US State'].isin(la_vars), 'US State'] = "Louisiana"
artists_df.loc[artists_df['US State'].isin(tn_vars), 'US State'] = "Tennessee"
artists_df.loc[artists_df['US State'].isin(mo_vars), 'US State'] = "Missouri"
artists_df.loc[artists_df['US State'].isin(vt_vars), 'US State'] = "Vermont"
artists_df.loc[artists_df['US State'].isin(ga_vars), 'US State'] = "Georgia"
artists_df.loc[artists_df['US State'].isin(wa_vars), 'US State'] = "Washington"
artists_df.loc[artists_df['US State'].isin(hw_vars), 'US State'] = "Hawaii"
artists_df.loc[artists_df['US State'].isin(null_vars), 'US State'] = None
artists_df.loc[artists_df['ArtistID'] == 'A25', 'US State'] = "South Carolina"



In [ ]:
artists_df["US State"].unique()

array(['Massachusetts', 'Georgia', 'Texas', 'Mississippi', 'Wisconsin',
       None, 'New York City', 'Pennsylvania', 'California', 'Maryland',
       'South Carolina', 'North Carolina', 'New Jersey', 'Louisiana',
       'D.C.', 'Michigan', 'Indiana', 'Florida', 'Oklahoma', 'Illinois',
       'Tennessee', 'Washington', 'Kentucky', 'Arizona', 'Maine',
       'Missouri', 'Utah', 'Virginia', 'Oregon', 'Alabama', 'Minnesota',
       'Nevada', 'Nebraska', 'Ohio', 'West Virginia', 'Iowa',
       'New Hampshire', 'Kansas', 'Hawaii', 'U.S.', 'Rhode Island',
       'Colorado', 'Arkansas', 'Connecticut', 'Vermont', 'New Mexico',
       'Delaware', 'North Dakota', 'Alaska', 'Wyoming'], dtype=object)

In [ ]:
artists_df["US State"].value_counts()

,count
US State,
California,263
New York City,189
Georgia,131
Texas,119
Tennessee,105
Florida,80
Illinois,54
Pennsylvania,51
Michigan,40


In [ ]:
artists_df.loc[:, "US State"] = artists_df["US State"].apply(lambda x: re.sub(r'[^\w\s]', '', x) if isinstance(x, str) else x)

In [ ]:
artists_df #2303x12

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist,US State
0,A0,Aerosmith,1,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith,Massachusetts
1,A1,Monica,1,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica,Georgia
2,A2,Jennifer Paige,1,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige,Georgia
3,A3,Usher,1,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher,Texas
4,A4,Brandy,1,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...
2298,A2298,Warren Zevon,0,NaN,United States,1947-01-24,1.0,1,https://en.wikipedia.org/wiki/Warren_Zevon,M,Warren Zevon,Illinois
2299,A2299,Bob Dylan,0,Robert Allen Zimmerman,United States,1941-05-24,1.0,1,https://en.wikipedia.org/wiki/Bob_Dylan,M,Bob Dylan,Minnesota
2300,A2300,Steely Dan,0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan,New York City
2301,A2301,Paul Simon,0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon,New York City


In [ ]:
artists_df.to_csv("artists_data_final_plusstates.csv", index=False)



In [ ]:
import pandas as pd
#quick fix here
artists_df= pd.read_csv("artists_data_final_plusstates2.csv")
artists_df #2305 rows × 12 columns

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist,US State
0,A0,Aerosmith,1.0,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith,Massachusetts
1,A1,Monica,1.0,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica,Georgia
2,A2,Jennifer Paige,1.0,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige,Georgia
3,A3,Usher,1.0,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher,Texas
4,A4,Brandy,1.0,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,A2300,Steely Dan,0.0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan,New York City
2301,A2301,Paul Simon,0.0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon,New York City
2302,A2302,Shelby Lynne,0.0,NaN,United States,1968-10-22,1.0,1,https://en.wikipedia.org/wiki/Shelby_Lynne,F,Shelby Lynne,Virginia
2303,A2303,JT,0.0,Jatavia Shakara Johnson,United States,1992-12-03,1.0,1,https://en.wikipedia.org/wiki/JT_(rapper),F,Jt,NaN


In [ ]:
artists_df.loc[artists_df['ArtistID']=='A2303', 'US State']="Florida"
artists_df.loc[artists_df['ArtistID']=='A2304', 'US State']="North Carolina"

In [ ]:
artists_df["US State"].value_counts()

,count
US State,
California,263
New York City,189
Georgia,131
Texas,119
Tennessee,105
Florida,81
Illinois,54
Pennsylvania,51
Michigan,40


In [ ]:
artists_df.to_csv("artists_data_FINAL.csv", index=False)
artists_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist,US State
0,A0,Aerosmith,1.0,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith,Massachusetts
1,A1,Monica,1.0,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica,Georgia
2,A2,Jennifer Paige,1.0,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige,Georgia
3,A3,Usher,1.0,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher,Texas
4,A4,Brandy,1.0,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,A2300,Steely Dan,0.0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan,New York City
2301,A2301,Paul Simon,0.0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon,New York City
2302,A2302,Shelby Lynne,0.0,NaN,United States,1968-10-22,1.0,1,https://en.wikipedia.org/wiki/Shelby_Lynne,F,Shelby Lynne,Virginia
2303,A2303,JT,0.0,Jatavia Shakara Johnson,United States,1992-12-03,1.0,1,https://en.wikipedia.org/wiki/JT_(rapper),F,Jt,Florida


In [4]:
import pandas as pd
artists_df= pd.read_csv("artists_data_FINAL.csv")


In [5]:
artists_df

,ArtistID,Main Artist,Spotify Found,Full Name,Country,Birth Date,Group Members,Found Wiki,Wiki URL,Gender,Cleaned Main Artist,US State
0,A0,Aerosmith,1.0,NaN,United States,NaN,5.0,1,https://en.wikipedia.org/wiki/Aerosmith,NaN,Aerosmith,Massachusetts
1,A1,Monica,1.0,Monica Denise Arnold,United States,1980-10-24,1.0,1,https://en.wikipedia.org/wiki/Monica_(singer),F,Monica,Georgia
2,A2,Jennifer Paige,1.0,NaN,United States,1973-09-03,1.0,1,https://en.wikipedia.org/wiki/Jennifer_Paige,F,Jennifer Paige,Georgia
3,A3,Usher,1.0,Usher Raymond IV[1][2],United States,1978-10-14,1.0,1,https://en.wikipedia.org/wiki/Usher_(musician),M,Usher,Texas
4,A4,Brandy,1.0,Brandy Rayana Norwood,United States,1979-02-11,1.0,1,https://en.wikipedia.org/wiki/Brandy_Norwood,F,Brandy,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...
2300,A2300,Steely Dan,0.0,NaN,United States,NaN,13.0,1,https://en.wikipedia.org/wiki/Steely_Dan,NaN,Steely Dan,New York City
2301,A2301,Paul Simon,0.0,NaN,United States,1941-10-13,1.0,1,https://en.wikipedia.org/wiki/Paul_Simon,M,Paul Simon,New York City
2302,A2302,Shelby Lynne,0.0,NaN,United States,1968-10-22,1.0,1,https://en.wikipedia.org/wiki/Shelby_Lynne,F,Shelby Lynne,Virginia
2303,A2303,JT,0.0,Jatavia Shakara Johnson,United States,1992-12-03,1.0,1,https://en.wikipedia.org/wiki/JT_(rapper),F,Jt,Florida


In [ ]:
# import plotly.express as px
# import pandas as pd

# # Example data with state abbreviations
# data = {
#     "State": ["CA", "NY", "TN", "TX"],  # Use state codes instead of full names
#     "Artist Count": [150, 120, 80, 90]
# }
# df = pd.DataFrame(data)

# # Create a choropleth map
# fig = px.choropleth(df,
#                     locations="State",
#                     locationmode="USA-states",  # Ensure it maps correctly
#                     color="Artist Count",
#                     scope="usa",
#                     color_continuous_scale="reds")

# fig.show()

